In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Charger les données
df = pd.read_excel('Data_09092025.xlsx')

In [3]:
df

,ID,Nom,Activité,Année,Liquidité générale,Liquidité immédiate,Taux de croissance du CA,Marge nette,Var Achats,Var CA&VarStock,REX/CA,IS/CA,RN/ACTIF,Ratio des immobilisations corporelles,Délai moyen de règlement des clients,Délai moyen de règlement des fournisseurs
0,9900223186066,STE ID 09900223186066,Activité 5,2022,3.61,2.11,-0.069,0.015,-0.0818,-0.0694,0.037,0.008,0.022,0.0055,118,62
1,9900223186066,STE ID 09900223186066,Activité 5,2024,5.66,3.86,-0.067,0.015,-0.0689,-0.0674,0.026,0.006,0.024,0.0222,98,29
2,9900223186066,STE ID 09900223186066,Activité 5,2021,5.11,3.50,0.009,0.013,0.0587,0.0094,0.036,0.008,0.018,0.0050,154,46
3,9900223186066,STE ID 09900223186066,Activité 5,2020,4.42,3.05,0.083,0.030,0.0535,0.0834,0.061,0.014,0.049,0.0052,124,38
4,9900223186066,STE ID 09900223186066,Activité 5,2023,3.41,2.52,0.319,0.011,0.3560,0.3188,0.033,0.006,0.021,0.0059,104,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249,99990226581981,STE ID 99990226581981,Activité 2,2022,2.30,2.28,-0.188,-7.004,-0.6864,-0.1876,-655.347,0.625,-0.001,0.0505,2008,28471
4250,99990226581981,STE ID 99990226581981,Activité 2,2023,2.04,2.03,-0.007,-16.805,3.6149,-0.0075,-728.239,1.017,-0.001,0.0505,2024,38194
4251,99990226581981,STE ID 99990226581981,Activité 2,2024,2.67,2.65,0.008,-37.986,0.3067,0.0083,-784.084,1.138,-0.003,0.0501,2007,42399
4252,99990226581981,STE ID 99990226581981,Activité 2,2020,2.72,2.68,0.022,-3.903,-0.5109,0.0218,-648.266,0.919,0.000,0.0528,2013,28463


In [4]:
#Etape de pretraitement des données

In [5]:
# Afficher les informations de base sur le dataset
print("INFORMATIONS GÉNÉRALES SUR LE DATASET")
print(f"Forme du dataset: {df.shape}")
print(f"Nombre d'entreprises: {df['Company'].nunique() if 'Company' in df.columns else 'Colonne Company non trouvée'}")
print(f"Nombre d'observations: {len(df)}")


INFORMATIONS GÉNÉRALES SUR LE DATASET
Forme du dataset: (4254, 16)
Nombre d'entreprises: Colonne Company non trouvée
Nombre d'observations: 4254


In [6]:
print("COLONNES DU DATASET")
print(df.columns.tolist())

COLONNES DU DATASET
['ID', 'Nom', 'Activité', 'Année', 'Liquidité générale', 'Liquidité immédiate', 'Taux de croissance du CA', 'Marge nette', 'Var Achats', 'Var CA&VarStock', 'REX/CA', 'IS/CA', 'RN/ACTIF', 'Ratio des immobilisations corporelles', 'Délai moyen de règlement des clients', 'Délai moyen de règlement des fournisseurs']


In [7]:
print("PREMIÈRES LIGNES du Dataset")
print(df.head())

PREMIÈRES LIGNES du Dataset
              ID                    Nom    Activité  Année  \
0  9900223186066  STE ID 09900223186066  Activité 5   2022   
1  9900223186066  STE ID 09900223186066  Activité 5   2024   
2  9900223186066  STE ID 09900223186066  Activité 5   2021   
3  9900223186066  STE ID 09900223186066  Activité 5   2020   
4  9900223186066  STE ID 09900223186066  Activité 5   2023   

   Liquidité générale  Liquidité immédiate  Taux de croissance du CA  \
0                3.61                 2.11                    -0.069   
1                5.66                 3.86                    -0.067   
2                5.11                 3.50                     0.009   
3                4.42                 3.05                     0.083   
4                3.41                 2.52                     0.319   

   Marge nette  Var Achats  Var CA&VarStock  REX/CA  IS/CA  RN/ACTIF  \
0        0.015     -0.0818          -0.0694   0.037  0.008     0.022   
1        0.015     -0.

In [8]:
print("INFORMATIONS SUR LES TYPES DE DONNÉES")
print(df.info())

print("STATISTIQUES DESCRIPTIVES")
print(df.describe())

INFORMATIONS SUR LES TYPES DE DONNÉES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4254 entries, 0 to 4253
Data columns (total 16 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   ID                                         4254 non-null   int64  
 1   Nom                                        4254 non-null   object 
 2   Activité                                   4254 non-null   object 
 3   Année                                      4254 non-null   int64  
 4   Liquidité générale                         4241 non-null   float64
 5   Liquidité immédiate                        4242 non-null   float64
 6   Taux de croissance du CA                   4056 non-null   float64
 7   Marge nette                                4088 non-null   float64
 8   Var Achats                                 4193 non-null   float64
 9   Var CA&VarStock                            4136 non-null  

In [ ]:
# Analyse plus détaillée des données
print("ANALYSE DÉTAILLÉE")

# Vérification des secteurs d'activité
print("Secteurs d'activité et observation (Activité):")
print(df['Activité'].value_counts())
print(f"Nombre de secteurs: {df['Activité'].nunique()}")

# Vérification des années
print("Répartition par année:")
print(df['Année'].value_counts().sort_index())

# Vérification de nombre d'entreprises uniques
print(f"\nNombre d'entreprises uniques (basé sur l'ID): {df['ID'].nunique()}")


# Identification des indicateurs financiers (colonnes numériques sauf ID et Année)
indicateurs_financiers = [col for col in df.columns if col not in ['ID', 'Nom', 'Activité', 'Année']]
print(f"INDICATEURS FINANCIERS IDENTIFIÉS ({len(indicateurs_financiers)})")
for i, ind in enumerate(indicateurs_financiers, 1):
    print(f"{i}. {ind}")



ANALYSE DÉTAILLÉE
Secteurs d'activité et observation (Activité):
Activité
Activité 5    1352
Activité 4    1052
Activité 2     907
Activité 6     599
Activité 1     274
Activité 3      70
Name: count, dtype: int64
Nombre de secteurs: 6
Répartition par année:
Année
2020    860
2021    863
2022    853
2023    857
2024    821
Name: count, dtype: int64

Nombre d'entreprises uniques (basé sur l'ID): 885
INDICATEURS FINANCIERS IDENTIFIÉS (12)
1. Liquidité générale
2. Liquidité immédiate
3. Taux de croissance du CA
4. Marge nette
5. Var Achats
6. Var CA&VarStock
7. REX/CA
8. IS/CA
9. RN/ACTIF
10. Ratio des immobilisations corporelles
11. Délai moyen de règlement des clients
12. Délai moyen de règlement des fournisseurs


In [9]:
# Vérification des valeurs manquantes
print("VALEURS MANQUANTES")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Colonne': missing_values.index,
    'Valeurs_manquantes': missing_values.values,
    'Pourcentage': missing_percentage.values
}).sort_values('Valeurs_manquantes', ascending=False)

print(missing_df[missing_df['Valeurs_manquantes'] > 0])


VALEURS MANQUANTES
                                  Colonne  Valeurs_manquantes  Pourcentage
11                                  IS/CA                 205     4.818994
6                Taux de croissance du CA                 198     4.654443
7                             Marge nette                 166     3.902210
10                                 REX/CA                 161     3.784673
9                         Var CA&VarStock                 118     2.773860
13  Ratio des immobilisations corporelles                  90     2.115656
8                              Var Achats                  61     1.433945
4                      Liquidité générale                  13     0.305595
5                     Liquidité immédiate                  12     0.282087
12                               RN/ACTIF                  11     0.258580


In [11]:
# Vérification des valeurs aberrantes
print("DÉTECTION DE VALEURS ABERRANTES par la méthode IQR")

# Identification des indicateurs financiers (colonnes numériques sauf ID et Année)
indicateurs_financiers = [col for col in df.columns if col not in ['ID', 'Nom', 'Activité', 'Année']]

for col in indicateurs_financiers[:5]:  # Premiers 5 indicateurs
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col].count()
    print(f"{col}: {outliers} valeurs aberrantes ({outliers/len(df)*100:.1f}%)")

DÉTECTION DE VALEURS ABERRANTES par la méthode IQR
Liquidité générale: 343 valeurs aberrantes (8.1%)
Liquidité immédiate: 299 valeurs aberrantes (7.0%)
Taux de croissance du CA: 520 valeurs aberrantes (12.2%)
Marge nette: 983 valeurs aberrantes (23.1%)
Var Achats: 607 valeurs aberrantes (14.3%)


In [12]:
#Identification des outliers par la fonction Isolation Forest
# Sélectionner uniquement les colonnes numériques
df_num = df.select_dtypes(include=['number'])

# Appliquer l'Isolation Forest uniquement sur les colonnes numériques
iso_forest = IsolationForest(contamination=0.1, random_state=42)
outlier_labels = iso_forest.fit_predict(df_num)

# Ajouter les labels d'anomalie au DataFrame original
df['Outlier'] = outlier_labels

# Afficher les outliers
outliers = df[df['Outlier'] == -1]
print("Outliers détectés :")
print(outliers)


Outliers détectés :
                  ID                    Nom    Activité  Année  \
18     9901223384014  STE ID 09901223384014  Activité 4   2021   
19     9901223384014  STE ID 09901223384014  Activité 4   2024   
46     9902226983013  STE ID 09902226983013  Activité 5   2023   
74     9905223383003  STE ID 09905223383003  Activité 2   2021   
75     9905223383003  STE ID 09905223383003  Activité 2   2020   
...              ...                    ...         ...    ...   
4249  99990226581981  STE ID 99990226581981  Activité 2   2022   
4250  99990226581981  STE ID 99990226581981  Activité 2   2023   
4251  99990226581981  STE ID 99990226581981  Activité 2   2024   
4252  99990226581981  STE ID 99990226581981  Activité 2   2020   
4253  99990226581981  STE ID 99990226581981  Activité 2   2021   

      Liquidité générale  Liquidité immédiate  Taux de croissance du CA  \
18                  1.06                 0.09                     2.495   
19                  1.40             

In [ ]:
#Analyse et nettoyage des outliers et valeurs manquantes


In [13]:
# 2. NETTOYAGE DES DONNÉES
print("\n🧹 1.2 NETTOYAGE DES DONNÉES")
print("-" * 40)

# 2.1 Analyse des valeurs manquantes
print("📊 ANALYSE DES VALEURS MANQUANTES:")
indicateurs_financiers = ['Liquidité générale', 'Liquidité immédiate', 'Taux de croissance du CA',
                          'Marge nette', 'Var Achats', 'Var CA&VarStock', 'REX/CA', 'IS/CA', 'RN/ACTIF', 'Ratio des immobilisations corporelles',
                          'Délai moyen de règlement des clients', 'Délai moyen de règlement des fournisseurs']
missing_analysis = pd.DataFrame({
    'Colonne': indicateurs_financiers,
    'Valeurs_manquantes': [df[col].isnull().sum() for col in indicateurs_financiers],
    'Pourcentage': [df[col].isnull().sum() / len(df) * 100 for col in indicateurs_financiers]
})

print(missing_analysis)

# 2.2 Gestion des valeurs manquantes
print("\n🔧 TRAITEMENT DES VALEURS MANQUANTES:")
df_clean = df.copy()

# Imputation par la médiane par secteur et année
for col in indicateurs_financiers:
    if df_clean[col].isnull().sum() > 0:
        # Imputation par médiane secteur/année
        df_clean[col] = df_clean.groupby(['Activité', 'Année'])[col].transform(
            lambda x: x.fillna(x.median())
        )

        # Si encore des valeurs manquantes, imputation par médiane globale du secteur
        df_clean[col] = df_clean.groupby('Activité')[col].transform(
            lambda x: x.fillna(x.median())
        )

        # En dernier recours, médiane globale
        df_clean[col].fillna(df_clean[col].median(), inplace=True)

        remaining_missing = df_clean[col].isnull().sum()
        print(f"✅ {col}: {remaining_missing} valeurs manquantes restantes")

print(f"\n✅ Total des valeurs manquantes après nettoyage: {df_clean.isnull().sum().sum()}")




🧹 1.2 NETTOYAGE DES DONNÉES
----------------------------------------
📊 ANALYSE DES VALEURS MANQUANTES:
                                      Colonne  Valeurs_manquantes  Pourcentage
0                          Liquidité générale                  13     0.305595
1                         Liquidité immédiate                  12     0.282087
2                    Taux de croissance du CA                 198     4.654443
3                                 Marge nette                 166     3.902210
4                                  Var Achats                  61     1.433945
5                             Var CA&VarStock                 118     2.773860
6                                      REX/CA                 161     3.784673
7                                       IS/CA                 205     4.818994
8                                    RN/ACTIF                  11     0.258580
9       Ratio des immobilisations corporelles                  90     2.115656
10       Délai moyen de règ

In [14]:
# 2.3 Détection et traitement des outliers par secteur
print("\n🔍 DÉTECTION DES OUTLIERS PAR SECTEUR:")

def detect_outliers_iqr(data, column, multiplier=1.5):
    """Détecte les outliers avec la méthode IQR"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    outliers = (data[column] < lower_bound) | (data[column] > upper_bound)
    return outliers, lower_bound, upper_bound

outliers_summary = []
df_outliers_treated = df_clean.copy()

for col in indicateurs_financiers:
    total_outliers = 0
    for secteur in df_clean['Activité'].unique():
        mask_secteur = df_clean['Activité'] == secteur
        data_secteur = df_clean[mask_secteur]

        outliers, lower, upper = detect_outliers_iqr(data_secteur, col)
        nb_outliers = outliers.sum()
        total_outliers += nb_outliers

        if nb_outliers > 0:
            # Winsorisation : remplacer les outliers par les percentiles 5% et 95%
            p5 = data_secteur[col].quantile(0.05)
            p95 = data_secteur[col].quantile(0.95)

            mask_outliers_low = (df_outliers_treated['Activité'] == secteur) & (df_outliers_treated[col] < lower)
            mask_outliers_high = (df_outliers_treated['Activité'] == secteur) & (df_outliers_treated[col] > upper)

            df_outliers_treated.loc[mask_outliers_low, col] = p5
            df_outliers_treated.loc[mask_outliers_high, col] = p95

    outliers_summary.append({
        'Indicateur': col,
        'Outliers_détectés': total_outliers,
        'Pourcentage': (total_outliers / len(df_clean)) * 100
    })

outliers_df = pd.DataFrame(outliers_summary)
print(outliers_df)

print("\n✅ Outliers traités par winsorisation (percentiles 5%-95%) par secteur")


🔍 DÉTECTION DES OUTLIERS PAR SECTEUR:
                                   Indicateur  Outliers_détectés  Pourcentage
0                          Liquidité générale                341     8.015985
1                         Liquidité immédiate                300     7.052186
2                    Taux de croissance du CA                489    11.495063
3                                 Marge nette                823    19.346497
4                                  Var Achats                481    11.307005
5                             Var CA&VarStock                494    11.612600
6                                      REX/CA                655    15.397273
7                                       IS/CA                492    11.565585
8                                    RN/ACTIF                489    11.495063
9       Ratio des immobilisations corporelles                145     3.408557
10       Délai moyen de règlement des clients                364     8.556653
11  Délai moyen de règlem

In [15]:
# 3. NORMALISATION PAR SECTEUR/ANNÉE
print("\n📏 1.3 NORMALISATION PAR SECTEUR/ANNÉE")
print("-" * 45)

df_normalized = df_outliers_treated.copy()

# 3.1 Standardisation Z-score par secteur
print("🎯 STANDARDISATION Z-SCORE PAR SECTEUR:")
for col in indicateurs_financiers:
    col_zscore = f"{col}_zscore"
    df_normalized[col_zscore] = df_normalized.groupby('Activité')[col].transform(
        lambda x: (x - x.mean()) / x.std()
    )
    print(f"✅ {col} → {col_zscore}")

# 3.2 Normalisation par quantiles (par secteur)
print("\n📊 NORMALISATION PAR QUANTILES (PAR SECTEUR):")
for col in indicateurs_financiers:
    col_quantile = f"{col}_quantile"
    df_normalized[col_quantile] = df_normalized.groupby('Activité')[col].transform(
        lambda x: x.rank(pct=True)
    )
    print(f"✅ {col} → {col_quantile}")

print(f"\n✅ Dataset enrichi: {df_normalized.shape[1]} colonnes (dont {len(indicateurs_financiers)*2} normalisées)")




📏 1.3 NORMALISATION PAR SECTEUR/ANNÉE
---------------------------------------------
🎯 STANDARDISATION Z-SCORE PAR SECTEUR:
✅ Liquidité générale → Liquidité générale_zscore
✅ Liquidité immédiate → Liquidité immédiate_zscore
✅ Taux de croissance du CA → Taux de croissance du CA_zscore
✅ Marge nette → Marge nette_zscore
✅ Var Achats → Var Achats_zscore
✅ Var CA&VarStock → Var CA&VarStock_zscore
✅ REX/CA → REX/CA_zscore
✅ IS/CA → IS/CA_zscore
✅ RN/ACTIF → RN/ACTIF_zscore
✅ Ratio des immobilisations corporelles → Ratio des immobilisations corporelles_zscore
✅ Délai moyen de règlement des clients → Délai moyen de règlement des clients_zscore
✅ Délai moyen de règlement des fournisseurs → Délai moyen de règlement des fournisseurs_zscore

📊 NORMALISATION PAR QUANTILES (PAR SECTEUR):
✅ Liquidité générale → Liquidité générale_quantile
✅ Liquidité immédiate → Liquidité immédiate_quantile
✅ Taux de croissance du CA → Taux de croissance du CA_quantile
✅ Marge nette → Marge nette_quantile
✅ Var Acha

In [16]:
# 4. CRÉATION DE NOUVEAUX INDICATEURS
print("\n🚀 1.4 CRÉATION DE NOUVEAUX INDICATEURS")
print("-" * 45)

# 4.1 DSO - DPO (Besoin en Fonds de Roulement)
df_normalized['DSO_DPO'] = df_normalized['Délai moyen de règlement des clients'] - df_normalized['Délai moyen de règlement des fournisseurs']
print("✅ DSO-DPO (Besoin en Fonds de Roulement proxy) créé")

# 4.2 Croissance du CA (variation annuelle par entreprise)
print("\n📈 CALCUL DE LA CROISSANCE CA:")
df_normalized = df_normalized.sort_values(['Nom', 'Année'])
df_normalized['Taux de croissance CA'] = df_normalized.groupby('Nom')['Année'].pct_change()

# Pour la première année de chaque entreprise, utiliser 0
df_normalized['Taux de croissance CA'].fillna(0, inplace=True)

# Utiliser les données disponibles comme Taux de croissance
# Calculer une croissance basée sur les ratios existants
df_normalized['Croissance_CA'] = (
  df_normalized['Marge nette'] * df_normalized['RN/ACTIF'] *
    (1 + df_normalized['Liquidité générale'] / 10)
).fillna(0)

print("✅ Croissance CA (proxy basé sur les ratios) créée")

# 4.3 Volatilité de la marge (écart-type mobile sur 3 ans)
print("\n📊 CALCUL DE LA VOLATILITÉ MARGE:")
df_normalized = df_normalized.sort_values(['Nom', 'Année'])
df_normalized['Volatilité_Marge'] = df_normalized.groupby('Nom')['Marge nette'].transform(
    lambda x: x.rolling(window=3, min_periods=1).std()
).fillna(0)

print("✅ Volatilité Marge (écart-type mobile 3 ans) créée")

# 4.4 Score de liquidité composite
df_normalized['Score_Liquidité'] = (
    df_normalized['Liquidité générale'] * 0.6 +
    (1 / (df_normalized['DSO_DPO'].abs() + 1)) * 0.4
)
print("✅ Score Liquidité composite créé")

# 4.5 Score de rentabilité composite
df_normalized['Score_Rentabilité'] = (
    df_normalized['Marge nette'] * 0.4 +
    df_normalized['RN/ACTIF'] * 0.6
)
print("✅ Score Rentabilité composite créé")

print(f"\n✅ Nouveaux indicateurs créés: 6 indicateurs supplémentaires")
print(f"📊 Dataset final enrichi: {df_normalized.shape[1]} colonnes")


🚀 1.4 CRÉATION DE NOUVEAUX INDICATEURS
---------------------------------------------
✅ DSO-DPO (Besoin en Fonds de Roulement proxy) créé

📈 CALCUL DE LA CROISSANCE CA:
✅ Croissance CA (proxy basé sur les ratios) créée

📊 CALCUL DE LA VOLATILITÉ MARGE:
✅ Volatilité Marge (écart-type mobile 3 ans) créée
✅ Score Liquidité composite créé
✅ Score Rentabilité composite créé

✅ Nouveaux indicateurs créés: 6 indicateurs supplémentaires
📊 Dataset final enrichi: 47 colonnes


In [17]:
# 5. CRÉATION DU FEATURE STORE
print("\n🏪 1.5 CRÉATION DU FEATURE STORE")
print("-" * 40)

# 5.1 Organisation des features par catégorie
feature_categories = {
    'Identifiants': ['Nom', 'Activité', 'Année'],

    'Indicateurs_Bruts': [
        'Liquidité_générale', 'Marge_nette', 'IS_CA', 'RN_ACTIF',
        'Délai_clients', 'Délai_fournisseurs', 'Ratio_immobilisations'
    ],

    'Indicateurs_Standardisés': [
        'Liquidité_générale_zscore', 'Marge_nette_zscore', 'IS_CA_zscore',
        'RN_ACTIF_zscore', 'Délai_clients_zscore', 'Délai_fournisseurs_zscore',
        'Ratio_immobilisations_zscore'
    ],

    'Indicateurs_Quantiles': [
        'Liquidité_générale_quantile', 'Marge_nette_quantile', 'IS_CA_quantile',
        'RN_ACTIF_quantile', 'Délai_myen_de_règlement_clients_quantile', 'Délai_moyen_de_règlement_fournisseurs_quantile',
        'Ratio_immobilisations_quantile'
    ],

    'Nouveaux_Indicateurs': [
        'DSO_DPO', 'Croissance_CA', 'Volatilité_Marge',
        'Score_Liquidité', 'Score_Rentabilité'
    ]
}

# 5.2 Création du Feature Store final
feature_store = df_normalized.copy()

# Ajout de métadonnées
feature_store['Date_Creation'] = pd.Timestamp.now()
feature_store['Version_Feature_Store'] = '1.0'

print("📊 STRUCTURE DU FEATURE STORE:")
for category, features in feature_categories.items():
    print(f"\n🏷️  {category}: {len(features)} features")
    for feature in features:
        print(f"   • {feature}")

print(f"\n✅ Feature Store créé avec {len(feature_store)} observations")
print(f"📈 Total features: {len(feature_store.columns)} colonnes")

# 5.3 Statistiques du Feature Store
print("\n📊 STATISTIQUES DU FEATURE STORE:")
print("-" * 40)

# Statistiques par secteur
stats_secteur = feature_store.groupby('Activité').agg({
    'Nom': 'nunique',
    'Score_Liquidité': ['mean', 'std'],
    'Score_Rentabilité': ['mean', 'std'],
    'DSO_DPO': 'mean',
    'Volatilité_Marge': 'mean'
}).round(3)

stats_secteur.columns = ['Nb_Entreprises', 'Liquidité_Moy', 'Liquidité_Std',
                        'Rentabilité_Moy', 'Rentabilité_Std', 'DSO_DPO_Moy', 'Volatilité_Moy']

print("🏭 STATISTIQUES PAR SECTEUR:")
print(stats_secteur)

# 5.4 Matrice de corrélation des nouveaux indicateurs
nouveaux_indicateurs = ['DSO_DPO', 'Taux de croissance CA', 'Volatilité_Marge',
                       'Score_Liquidité', 'Score_Rentabilité']

correlation_matrix = feature_store[nouveaux_indicateurs].corr().round(3)
print(f"\n🔗 MATRICE DE CORRÉLATION (NOUVEAUX INDICATEURS):")
print(correlation_matrix)

print("\n✅ ÉTAPE 1 TERMINÉE - FEATURE STORE PRÊT")
print("=" * 50)
print(f"📊 {len(feature_store)} observations traitées")
print(f"🏢 {feature_store['Nom'].nunique()} entreprises uniques")
print(f"🏭 {feature_store['Activité'].nunique()} secteurs d'activité")
print(f"📅 {feature_store['Année'].nunique()} années (2020-2024)")
print(f"📈 {len(feature_store.columns)} features totales")


🏪 1.5 CRÉATION DU FEATURE STORE
----------------------------------------
📊 STRUCTURE DU FEATURE STORE:

🏷️  Identifiants: 3 features
   • Nom
   • Activité
   • Année

🏷️  Indicateurs_Bruts: 7 features
   • Liquidité_générale
   • Marge_nette
   • IS_CA
   • RN_ACTIF
   • Délai_clients
   • Délai_fournisseurs
   • Ratio_immobilisations

🏷️  Indicateurs_Standardisés: 7 features
   • Liquidité_générale_zscore
   • Marge_nette_zscore
   • IS_CA_zscore
   • RN_ACTIF_zscore
   • Délai_clients_zscore
   • Délai_fournisseurs_zscore
   • Ratio_immobilisations_zscore

🏷️  Indicateurs_Quantiles: 7 features
   • Liquidité_générale_quantile
   • Marge_nette_quantile
   • IS_CA_quantile
   • RN_ACTIF_quantile
   • Délai_myen_de_règlement_clients_quantile
   • Délai_moyen_de_règlement_fournisseurs_quantile
   • Ratio_immobilisations_quantile

🏷️  Nouveaux_Indicateurs: 5 features
   • DSO_DPO
   • Croissance_CA
   • Volatilité_Marge
   • Score_Liquidité
   • Score_Rentabilité

✅ Feature Store créé a

In [18]:
 #6. EXPORT DU FEATURE STORE POUR SAUVEGARDE
print("\n💾 1.6 EXPORT ET SAUVEGARDE DU FEATURE STORE")
print("-" * 50)

# Sauvegarde en CSV pour utilisation ultérieure
feature_store.to_csv('Feature_Store_Carte_Risque_v1.csv', index=False)
print("✅ Feature Store sauvegardé: 'Feature_Store_Carte_Risque_v1.csv'")

# Création d'un dictionnaire de métadonnées
metadata = {
    'Date_Creation': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Version': '1.0',
    'Nombre_Observations': len(feature_store),
    'Nombre_Entreprises': feature_store['Nom'].nunique(),
    'Nombre_Secteurs': feature_store['Activité'].nunique(),
    'Periode': f"{feature_store['Année'].min()}-{feature_store['Année'].max()}",
    'Nombre_Features_Total': len(feature_store.columns),
    'Features_Categories': feature_categories,
    'Traitements_Appliques': [
        'Imputation valeurs manquantes par médiane secteur/année',
        'Winsorisation outliers (percentiles 5%-95%) par secteur',
        'Standardisation Z-score par secteur',
        'Normalisation quantiles par secteur',
        'Création indicateurs composites (DSO-DPO, Scores, Volatilité)'
    ]
}

# Affichage du résumé final
print("\n📋 RÉSUMÉ FINAL DE L'ÉTAPE 1:")
print("=" * 40)
for key, value in metadata.items():
    if key != 'Features_Categories' and key != 'Traitements_Appliques':
        print(f"{key.replace('_', ' ')}: {value}")

print("\n🔧 TRAITEMENTS APPLIQUÉS:")
for i, traitement in enumerate(metadata['Traitements_Appliques'], 1):
    print(f"{i}. {traitement}")

print("\n✅ ÉTAPE 1 COMPLÈTEMENT TERMINÉE")
print("🎯 Feature Store opérationnel pour la Carte de Risque IA")
print("📊 Données nettoyées, normalisées et enrichies")
print("🚀 Prêt pour l'étape 2: Pipeline de Data Engineering")

# Aperçu final des données clés
print(f"\n📈 APERÇU DES KPI CLÉS (PAR SECTEUR):")
kpi_summary = feature_store.groupby('Activité').agg({
    'Score_Liquidité': 'mean',
    'Score_Rentabilité': 'mean',
    'DSO_DPO': 'mean',
    'Volatilité_Marge': 'mean'
}).round(3)

kpi_summary.columns = ['Liquidité_Moy', 'Rentabilité_Moy', 'BFR_Proxy_Moy', 'Volatilité_Moy']
print(kpi_summary)


💾 1.6 EXPORT ET SAUVEGARDE DU FEATURE STORE
--------------------------------------------------
✅ Feature Store sauvegardé: 'Feature_Store_Carte_Risque_v1.csv'

📋 RÉSUMÉ FINAL DE L'ÉTAPE 1:
Date Creation: 2025-10-31 22:54:58
Version: 1.0
Nombre Observations: 4254
Nombre Entreprises: 885
Nombre Secteurs: 6
Periode: 2020-2024
Nombre Features Total: 49

🔧 TRAITEMENTS APPLIQUÉS:
1. Imputation valeurs manquantes par médiane secteur/année
2. Winsorisation outliers (percentiles 5%-95%) par secteur
3. Standardisation Z-score par secteur
4. Normalisation quantiles par secteur
5. Création indicateurs composites (DSO-DPO, Scores, Volatilité)

✅ ÉTAPE 1 COMPLÈTEMENT TERMINÉE
🎯 Feature Store opérationnel pour la Carte de Risque IA
📊 Données nettoyées, normalisées et enrichies
🚀 Prêt pour l'étape 2: Pipeline de Data Engineering

📈 APERÇU DES KPI CLÉS (PAR SECTEUR):
            Liquidité_Moy  Rentabilité_Moy  BFR_Proxy_Moy  Volatilité_Moy
Activité                                                      

In [19]:
print("🎯 ÉTAPES 2 & 3: SCORING AVANCÉ + DÉTECTION D'ANOMALIES")
print("=" * 60)

# Chargement du Feature Store créé à l'étape 1
try:
    feature_store = pd.read_csv('Feature_Store_Carte_Risque_IA_v1.csv')
    print(f"✅ Feature Store chargé: {feature_store.shape}")
except:
    print("⚠️ Rechargement des données depuis l'Excel...")
    # Si le CSV n'existe pas, recréer rapidement le feature store
    df_raw = pd.read_excel('Data_09092025.xlsx')
    feature_store = df_raw[['Nom', 'Activité', 'Année', 'Liquidité générale', 'Marge nette',
                           'IS/CA', 'RN/ACTIF', 'Délai moyen de règlement des clients',
                           'Délai moyen de règlement des fournisseurs',
                           'Ratio des immobilisations corporelles']].copy()

    feature_store.columns = ['Nom', 'Activité', 'Année', 'Liquidité_générale', 'Marge_nette',
                            'IS_CA', 'RN_ACTIF', 'Délai_clients', 'Délai_fournisseurs',
                            'Ratio_immobilisations']

    # Nettoyage rapide
    for col in ['Liquidité_générale', 'Marge_nette', 'IS_CA', 'RN_ACTIF', 'Ratio_immobilisations']:
        feature_store[col].fillna(feature_store[col].median(), inplace=True)

    # Création des indicateurs de base
    feature_store['DSO_DPO'] = feature_store['Délai_clients'] - feature_store['Délai_fournisseurs']
    feature_store['Score_Liquidité'] = feature_store['Liquidité_générale'] * 0.6 + (1 / (feature_store['DSO_DPO'].abs() + 1)) * 0.4
    feature_store['Score_Rentabilité'] = feature_store['Marge_nette'] * 0.4 + feature_store['RN_ACTIF'] * 0.6

print("\n📊 DONNÉES PRÊTES POUR LE SCORING")
print(f"Dimensions: {feature_store.shape}")
print(f"Entreprises: {feature_store['Nom'].nunique()}")
print(f"Secteurs: {feature_store['Activité'].nunique()}")
print(f"Années: {feature_store['Année'].nunique()}")

🎯 ÉTAPES 2 & 3: SCORING AVANCÉ + DÉTECTION D'ANOMALIES
⚠️ Rechargement des données depuis l'Excel...

📊 DONNÉES PRÊTES POUR LE SCORING
Dimensions: (4254, 13)
Entreprises: 885
Secteurs: 6
Années: 5


In [20]:
# Chargement des données pour les étapes 2 et 3
try:
    feature_store = pd.read_csv('Feature_Store_Carte_Risque_v1.csv')
    print(f"✅ Feature Store chargé: {feature_store.shape}")
except:
    print("⚠️ Rechargement des données depuis l'Excel...")
    # Recréation rapide du feature store
    df_raw = pd.read_excel('Data_09092025.xlsx')
    feature_store = df_raw[['Nom', 'Activité', 'Année', 'Liquidité générale', 'Marge nette',
                           'IS/CA', 'RN/ACTIF', 'Délai moyen de règlement des clients',
                           'Délai moyen de règlement des fournisseurs',
                           'Ratio des immobilisations corporelles']].copy()

    feature_store.columns = ['Nom', 'Activité', 'Année', 'Liquidité_générale', 'Marge_nette',
                            'IS_CA', 'RN_ACTIF', 'Délai_clients', 'Délai_fournisseurs',
                            'Ratio_immobilisations']

    # Nettoyage rapide des valeurs manquantes
    for col in ['Liquidité_générale', 'Marge_nette', 'IS_CA', 'RN_ACTIF', 'Ratio_immobilisations']:
        feature_store[col].fillna(feature_store[col].median(), inplace=True)

    # Création des indicateurs de base nécessaires
    feature_store['DSO_DPO'] = feature_store['Délai_clients'] - feature_store['Délai_fournisseurs']
    feature_store['Score_Liquidité'] = (feature_store['Liquidité_générale'] * 0.6 +
                                       (1 / (feature_store['DSO_DPO'].abs() + 1)) * 0.4)
    feature_store['Score_Rentabilité'] = (feature_store['Marge_nette'] * 0.4 +
                                         feature_store['RN_ACTIF'] * 0.6)

print("\n📊 DONNÉES PRÊTES POUR LE SCORING AVANCÉ")
print(f"Dimensions: {feature_store.shape}")
print(f"Entreprises: {feature_store['Nom'].nunique()}")
print(f"Secteurs: {feature_store['Activité'].nunique()}")
print(f"Années: {feature_store['Année'].min()}-{feature_store['Année'].max()}")

# Vérification des colonnes disponibles
print(f"\nColonnes disponibles: {list(feature_store.columns)}")

✅ Feature Store chargé: (4254, 49)

📊 DONNÉES PRÊTES POUR LE SCORING AVANCÉ
Dimensions: (4254, 49)
Entreprises: 885
Secteurs: 6
Années: 2020-2024

Colonnes disponibles: ['ID', 'Nom', 'Activité', 'Année', 'Liquidité générale', 'Liquidité immédiate', 'Taux de croissance du CA', 'Marge nette', 'Var Achats', 'Var CA&VarStock', 'REX/CA', 'IS/CA', 'RN/ACTIF', 'Ratio des immobilisations corporelles', 'Délai moyen de règlement des clients', 'Délai moyen de règlement des fournisseurs', 'Outlier', 'Liquidité générale_zscore', 'Liquidité immédiate_zscore', 'Taux de croissance du CA_zscore', 'Marge nette_zscore', 'Var Achats_zscore', 'Var CA&VarStock_zscore', 'REX/CA_zscore', 'IS/CA_zscore', 'RN/ACTIF_zscore', 'Ratio des immobilisations corporelles_zscore', 'Délai moyen de règlement des clients_zscore', 'Délai moyen de règlement des fournisseurs_zscore', 'Liquidité générale_quantile', 'Liquidité immédiate_quantile', 'Taux de croissance du CA_quantile', 'Marge nette_quantile', 'Var Achats_quantile'

In [21]:
# ÉTAPE 2: SCORING AVANCÉ
print("\n🎯 ÉTAPE 2: SCORING AVANCÉ")
print("=" * 40)

# 2.1 SCORING MÉTIER PONDÉRÉ
print("\n📊 2.1 SCORING MÉTIER PONDÉRÉ")
print("-" * 35)

# Définition des pondérations selon la mission
ponderations = {
    'Liquidité': 0.25,      # 25%
    'Rentabilité': 0.25,    # 25%
    'BFR': 0.20,           # 20% (DSO-DPO)
    'Fiscalité': 0.15,     # 15% (IS/CA)
    'Structure': 0.15      # 15% (Ratio immobilisations)
}

print("💰 PONDÉRATIONS APPLIQUÉES:")
for dimension, poids in ponderations.items():
    print(f"   • {dimension}: {poids*100}%")

# Normalisation des indicateurs pour le scoring (0-10)
def normalize_to_score(series, inverse=False):
    """Normalise une série entre 0 et 10 (inverse si plus petit = meilleur)"""
    if inverse:
        # Pour les indicateurs où plus petit = meilleur (ex: délais)
        normalized = 10 * (1 - (series - series.min()) / (series.max() - series.min() + 1e-8))
    else:
        # Pour les indicateurs où plus grand = meilleur (ex: liquidité)
        normalized = 10 * (series - series.min()) / (series.max() - series.min() + 1e-8)
    return normalized.fillna(5)  # Valeur neutre pour les NaN

# Calcul des scores par dimension
print("\n🔢 CALCUL DES SCORES PAR DIMENSION:")

# Score Liquidité (25%) - Plus élevé = meilleur
feature_store['Score_Liquidité_Norm'] = normalize_to_score(
    feature_store['Liquidité générale'], inverse=False
)
print(f"✅ Score Liquidité: moyenne = {feature_store['Score_Liquidité_Norm'].mean():.2f}")

# Score Rentabilité (25%) - Plus élevé = meilleur
feature_store['Score_Rentabilité_Norm'] = normalize_to_score(
    feature_store['RN/ACTIF'], inverse=False
)
print(f"✅ Score Rentabilité: moyenne = {feature_store['Score_Rentabilité_Norm'].mean():.2f}")

# Score BFR (20%) - DSO-DPO plus faible = meilleur
feature_store['Score_BFR'] = normalize_to_score(
    feature_store['DSO_DPO'].abs(), inverse=True
)
print(f"✅ Score BFR: moyenne = {feature_store['Score_BFR'].mean():.2f}")

# Score Fiscalité (15%) - IS/CA plus faible = meilleur (optimisation fiscale)
feature_store['Score_Fiscalité'] = normalize_to_score(
    feature_store['IS/CA'], inverse=True
)
print(f"✅ Score Fiscalité: moyenne = {feature_store['Score_Fiscalité'].mean():.2f}")

# Score Structure (15%) - Ratio immobilisations équilibré = meilleur
# Créer un score basé sur l'écart à une valeur optimale (ex: 0.1 = 10%)
optimal_ratio = 0.1
feature_store['Score_Structure'] = normalize_to_score(
    abs(feature_store['Ratio des immobilisations corporelles'] - optimal_ratio), inverse=True
)
print(f"✅ Score Structure: moyenne = {feature_store['Score_Structure'].mean():.2f}")

# SCORE MÉTIER FINAL PONDÉRÉ
feature_store['Score_Métier'] = (
    feature_store['Score_Liquidité_Norm'] * ponderations['Liquidité'] +
    feature_store['Score_Rentabilité_Norm'] * ponderations['Rentabilité'] +
    feature_store['Score_BFR'] * ponderations['BFR'] +
    feature_store['Score_Fiscalité'] * ponderations['Fiscalité'] +
    feature_store['Score_Structure'] * ponderations['Structure']
)

print(f"\n🎯 SCORE MÉTIER FINAL:")
print(f"   • Moyenne: {feature_store['Score_Métier'].mean():.2f}/10")
print(f"   • Médiane: {feature_store['Score_Métier'].median():.2f}/10")
print(f"   • Écart-type: {feature_store['Score_Métier'].std():.2f}")
print(f"   • Min-Max: [{feature_store['Score_Métier'].min():.2f} - {feature_store['Score_Métier'].max():.2f}]")

# Distribution des scores par décile
deciles = pd.qcut(feature_store['Score_Métier'], 10, labels=False) + 1
print(f"\n📊 DISTRIBUTION PAR DÉCILES:")
decile_stats = pd.DataFrame({
    'Décile': range(1, 11),
    'Nombre': deciles.value_counts().sort_index().values,
    'Pourcentage': (deciles.value_counts().sort_index().values / len(feature_store) * 100).round(1)
})
print(decile_stats)


🎯 ÉTAPE 2: SCORING AVANCÉ

📊 2.1 SCORING MÉTIER PONDÉRÉ
-----------------------------------
💰 PONDÉRATIONS APPLIQUÉES:
   • Liquidité: 25.0%
   • Rentabilité: 25.0%
   • BFR: 20.0%
   • Fiscalité: 15.0%
   • Structure: 15.0%

🔢 CALCUL DES SCORES PAR DIMENSION:
✅ Score Liquidité: moyenne = 1.74
✅ Score Rentabilité: moyenne = 4.59
✅ Score BFR: moyenne = 9.45
✅ Score Fiscalité: moyenne = 6.78
✅ Score Structure: moyenne = 5.63

🎯 SCORE MÉTIER FINAL:
   • Moyenne: 5.33/10
   • Médiane: 5.38/10
   • Écart-type: 0.89
   • Min-Max: [1.46 - 8.73]

📊 DISTRIBUTION PAR DÉCILES:
   Décile  Nombre  Pourcentage
0       1     426         10.0
1       2     425         10.0
2       3     425         10.0
3       4     426         10.0
4       5     425         10.0
5       6     425         10.0
6       7     426         10.0
7       8     425         10.0
8       9     425         10.0
9      10     426         10.0


In [22]:
# 2.2 MACHINE LEARNING SUPERVISÉ
print("\n🤖 2.2 MACHINE LEARNING SUPERVISÉ")
print("-" * 35)

# Création des labels pour l'apprentissage supervisé
# Stratégie: Utiliser le score métier pour créer des classes "Risque Haut vs Bas"
threshold_risque = feature_store['Score_Métier'].quantile(0.3)  # 30% plus bas = Risque Haut
print(f"🎯 Seuil de classification: {threshold_risque:.2f}")

# Création des labels binaires
feature_store['Risque_Label'] = (feature_store['Score_Métier'] <= threshold_risque).astype(int)
labels_count = feature_store['Risque_Label'].value_counts()
print(f"📊 Distribution des labels:")
print(f"   • Risque Bas (0): {labels_count[0]} ({labels_count[0]/len(feature_store)*100:.1f}%)")
print(f"   • Risque Haut (1): {labels_count[1]} ({labels_count[1]/len(feature_store)*100:.1f}%)")

# Préparation des features pour ML
features_ml = [
    'Liquidité générale', 'Marge nette', 'RN/ACTIF', 'IS/CA',
    'Ratio des immobilisations corporelles', 'DSO_DPO', 'Délai moyen de règlement des clients', 'Délai moyen de règlement des fournisseurs'
]

X = feature_store[features_ml].copy()
y = feature_store['Risque_Label'].copy()

# Gestion des valeurs manquantes et infinis
for col in X.columns:
    X[col] = X[col].replace([np.inf, -np.inf], np.nan)
    X[col] = X[col].fillna(X[col].median())

print(f"\n🔧 Features préparées: {X.shape[1]} variables")
print(f"📊 Échantillon: {X.shape[0]} observations")

# Standardisation des features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=features_ml)

# Division train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y, test_size=0.3, random_state=42, stratify=y
)

print(f"✅ Jeu d'entraînement: {X_train.shape[0]} observations")
print(f"✅ Jeu de test: {X_test.shape[0]} observations")

# MODÈLE 1: RANDOM FOREST
print(f"\n🌲 ENTRAÎNEMENT RANDOM FOREST:")
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    class_weight='balanced'
)

rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_proba = rf_model.predict_proba(X_test)[:, 1]

# Métriques Random Forest
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_auc = roc_auc_score(y_test, rf_proba)

print(f"✅ Accuracy: {rf_accuracy:.3f}")
print(f"✅ AUC-ROC: {rf_auc:.3f}")

# Cross-validation
cv_scores = cross_val_score(rf_model, X_scaled_df, y, cv=5, scoring='roc_auc')
print(f"✅ CV AUC moyenne: {cv_scores.mean():.3f} (+/- {cv_scores.std()*2:.3f})")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': features_ml,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n📊 IMPORTANCE DES FEATURES (Top 5):")
for i, row in feature_importance.head().iterrows():
    print(f"   • {row['Feature']}: {row['Importance']:.3f}")

# Prédictions ML sur tout le dataset
X_all_scaled = scaler.transform(X)
feature_store['Score_ML'] = rf_model.predict_proba(X_all_scaled)[:, 1] * 10  # Conversion 0-10

print(f"\n🎯 SCORE ML FINAL:")
print(f"   • Moyenne: {feature_store['Score_ML'].mean():.2f}/10")
print(f"   • Médiane: {feature_store['Score_ML'].median():.2f}/10")
print(f"   • Min-Max: [{feature_store['Score_ML'].min():.2f} - {feature_store['Score_ML'].max():.2f}]")


🤖 2.2 MACHINE LEARNING SUPERVISÉ
-----------------------------------
🎯 Seuil de classification: 4.94
📊 Distribution des labels:
   • Risque Bas (0): 2978 (70.0%)
   • Risque Haut (1): 1276 (30.0%)

🔧 Features préparées: 8 variables
📊 Échantillon: 4254 observations
✅ Jeu d'entraînement: 2977 observations
✅ Jeu de test: 1277 observations

🌲 ENTRAÎNEMENT RANDOM FOREST:
✅ Accuracy: 0.952
✅ AUC-ROC: 0.991
✅ CV AUC moyenne: 0.986 (+/- 0.007)

📊 IMPORTANCE DES FEATURES (Top 5):
   • Ratio des immobilisations corporelles: 0.222
   • RN/ACTIF: 0.209
   • Marge nette: 0.157
   • Liquidité générale: 0.123
   • DSO_DPO: 0.089

🎯 SCORE ML FINAL:
   • Moyenne: 3.32/10
   • Médiane: 1.25/10
   • Min-Max: [0.01 - 10.00]


In [23]:
# 2.3 SCORE HYBRIDE FINAL
print("\n🔀 2.3 SCORE HYBRIDE FINAL")
print("-" * 30)

# Combinaison Score Métier + Score ML
# Pondération: 60% Métier + 40% ML (approche équilibrée règles + IA)
poids_metier = 0.6
poids_ml = 0.4

feature_store['Score_Hybride'] = (
    feature_store['Score_Métier'] * poids_metier +
    feature_store['Score_ML'] * poids_ml
)

print(f"⚖️  PONDÉRATION HYBRIDE:")
print(f"   • Score Métier: {poids_metier*100}%")
print(f"   • Score ML: {poids_ml*100}%")

print(f"\n🎯 SCORE HYBRIDE FINAL:")
print(f"   • Moyenne: {feature_store['Score_Hybride'].mean():.2f}/10")
print(f"   • Médiane: {feature_store['Score_Hybride'].median():.2f}/10")
print(f"   • Écart-type: {feature_store['Score_Hybride'].std():.2f}")
print(f"   • Min-Max: [{feature_store['Score_Hybride'].min():.2f} - {feature_store['Score_Hybride'].max():.2f}]")

# Classification finale en catégories de risque
def classify_risk(score):
    if score <= 3:
        return "Très Haut"
    elif score <= 5:
        return "Haut"
    elif score <= 7:
        return "Modéré"
    else:
        return "Faible"

feature_store['Catégorie_Risque'] = feature_store['Score_Hybride'].apply(classify_risk)

# Distribution des catégories de risque
risk_distribution = feature_store['Catégorie_Risque'].value_counts()
print(f"\n📊 DISTRIBUTION DES CATÉGORIES DE RISQUE:")
for category, count in risk_distribution.items():
    percentage = count / len(feature_store) * 100
    print(f"   • {category}: {count} ({percentage:.1f}%)")

# Analyse par secteur
print(f"\n🏭 ANALYSE PAR SECTEUR:")
secteur_risk = feature_store.groupby('Activité').agg({
    'Score_Hybride': ['mean', 'std'],
    'Catégorie_Risque': lambda x: x.value_counts().index[0]  # Mode
}).round(2)

secteur_risk.columns = ['Score_Moyen', 'Écart_Type', 'Catégorie_Dominante']
print(secteur_risk)



🔀 2.3 SCORE HYBRIDE FINAL
------------------------------
⚖️  PONDÉRATION HYBRIDE:
   • Score Métier: 60.0%
   • Score ML: 40.0%

🎯 SCORE HYBRIDE FINAL:
   • Moyenne: 4.53/10
   • Médiane: 4.02/10
   • Écart-type: 1.12
   • Min-Max: [3.09 - 6.81]

📊 DISTRIBUTION DES CATÉGORIES DE RISQUE:
   • Haut: 2834 (66.6%)
   • Modéré: 1420 (33.4%)

🏭 ANALYSE PAR SECTEUR:
            Score_Moyen  Écart_Type Catégorie_Dominante
Activité                                               
Activité 1         4.72        1.16                Haut
Activité 2         4.85        1.15                Haut
Activité 3         4.44        0.93                Haut
Activité 4         5.31        1.01              Modéré
Activité 5         4.01        0.83                Haut
Activité 6         3.78        0.67                Haut


In [24]:
# ÉTAPE 3: DÉTECTION D'ANOMALIES
print("\n🔍 ÉTAPE 3: DÉTECTION D'ANOMALIES")
print("=" * 40)

# Préparation des données pour la détection d'anomalies
features_anomaly = [
    'Liquidité générale', 'Marge nette', 'RN/ACTIF', 'IS/CA',
    'Ratio des immobilisations corporelles', 'DSO_DPO', 'Score_Métier', 'Score_ML'
]

X_anomaly = feature_store[features_anomaly].copy()

# Nettoyage pour la détection d'anomalies
for col in X_anomaly.columns:
    X_anomaly[col] = X_anomaly[col].replace([np.inf, -np.inf], np.nan)
    X_anomaly[col] = X_anomaly[col].fillna(X_anomaly[col].median())

# Standardisation
scaler_anomaly = StandardScaler()
X_anomaly_scaled = scaler_anomaly.fit_transform(X_anomaly)

print(f"🔧 Features pour anomalies: {len(features_anomaly)} variables")
print(f"📊 Données préparées: {X_anomaly_scaled.shape[0]} observations")

# 3.1 ISOLATION FOREST
print(f"\n🌲 3.1 ISOLATION FOREST")
print("-" * 25)

# Paramètres optimisés pour la finance
iso_forest = IsolationForest(
    contamination=0.1,  # 10% d'anomalies attendues
    random_state=42,
    n_estimators=100
)

iso_predictions = iso_forest.fit_predict(X_anomaly_scaled)
iso_scores = iso_forest.decision_function(X_anomaly_scaled)

# Conversion: -1 (anomalie) -> 1, +1 (normal) -> 0
iso_anomalies = (iso_predictions == -1).astype(int)
feature_store['Iso_Forest_Anomaly'] = iso_anomalies
feature_store['Iso_Forest_Score'] = iso_scores

nb_anomalies_iso = iso_anomalies.sum()
print(f"✅ Anomalies détectées: {nb_anomalies_iso} ({nb_anomalies_iso/len(feature_store)*100:.1f}%)")
print(f"✅ Score moyen: {iso_scores.mean():.3f}")
print(f"✅ Seuil de détection: {iso_forest.offset_:.3f}")

# 3.2 LOCAL OUTLIER FACTOR (LOF)
print(f"\n📊 3.2 LOCAL OUTLIER FACTOR (LOF)")
print("-" * 35)

# LOF avec paramètres adaptés
lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination=0.1,
    novelty=False
)

lof_predictions = lof.fit_predict(X_anomaly_scaled)
lof_scores = lof.negative_outlier_factor_

# Conversion: -1 (anomalie) -> 1, +1 (normal) -> 0
lof_anomalies = (lof_predictions == -1).astype(int)
feature_store['LOF_Anomaly'] = lof_anomalies
feature_store['LOF_Score'] = lof_scores

nb_anomalies_lof = lof_anomalies.sum()
print(f"✅ Anomalies détectées: {nb_anomalies_lof} ({nb_anomalies_lof/len(feature_store)*100:.1f}%)")
print(f"✅ Score LOF moyen: {lof_scores.mean():.3f}")
print(f"✅ Score LOF min: {lof_scores.min():.3f}")

# 3.3 DBSCAN (CLUSTERING)
print(f"\n🔗 3.3 DBSCAN CLUSTERING")
print("-" * 25)

# DBSCAN avec paramètres optimisés
dbscan = DBSCAN(
    eps=0.5,        # Distance epsilon
    min_samples=10  # Minimum d'échantillons pour un cluster
)

dbscan_labels = dbscan.fit_predict(X_anomaly_scaled)

# Cluster -1 = anomalies (points non assignés)
dbscan_anomalies = (dbscan_labels == -1).astype(int)
feature_store['DBSCAN_Anomaly'] = dbscan_anomalies

nb_anomalies_dbscan = dbscan_anomalies.sum()
nb_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)

print(f"✅ Anomalies détectées: {nb_anomalies_dbscan} ({nb_anomalies_dbscan/len(feature_store)*100:.1f}%)")
print(f"✅ Nombre de clusters: {nb_clusters}")
print(f"✅ Points dans clusters: {len(feature_store) - nb_anomalies_dbscan}")

# Distribution des clusters
if nb_clusters > 0:
    cluster_sizes = pd.Series(dbscan_labels[dbscan_labels != -1]).value_counts().sort_index()
    print(f"✅ Taille des clusters: {cluster_sizes.values}")

print(f"\n📊 COMPARAISON DES 3 MÉTHODES:")
comparison_df = pd.DataFrame({
    'Méthode': ['Isolation Forest', 'LOF', 'DBSCAN'],
    'Nb_Anomalies': [nb_anomalies_iso, nb_anomalies_lof, nb_anomalies_dbscan],
    'Pourcentage': [
        nb_anomalies_iso/len(feature_store)*100,
        nb_anomalies_lof/len(feature_store)*100,
        nb_anomalies_dbscan/len(feature_store)*100
    ]
}).round(1)

print(comparison_df)


🔍 ÉTAPE 3: DÉTECTION D'ANOMALIES
🔧 Features pour anomalies: 8 variables
📊 Données préparées: 4254 observations

🌲 3.1 ISOLATION FOREST
-------------------------
✅ Anomalies détectées: 426 (10.0%)
✅ Score moyen: 0.103
✅ Seuil de détection: -0.547

📊 3.2 LOCAL OUTLIER FACTOR (LOF)
-----------------------------------
✅ Anomalies détectées: 426 (10.0%)
✅ Score LOF moyen: -1.127
✅ Score LOF min: -4.713

🔗 3.3 DBSCAN CLUSTERING
-------------------------
✅ Anomalies détectées: 1283 (30.2%)
✅ Nombre de clusters: 14
✅ Points dans clusters: 2971
✅ Taille des clusters: [2717   41   28   10   11   11   30   25   12   35   10   21   10   10]

📊 COMPARAISON DES 3 MÉTHODES:
            Méthode  Nb_Anomalies  Pourcentage
0  Isolation Forest           426         10.0
1               LOF           426         10.0
2            DBSCAN          1283         30.2


In [25]:
# 3.4 INDICE D'ANOMALIE CONSOLIDÉ
print("\n🔀 3.4 INDICE D'ANOMALIE CONSOLIDÉ")
print("-" * 35)

# --- Re-run the anomaly detection models to ensure scores are available ---
# Preparation for anomaly detection
features_anomaly = [
    'Liquidité générale', 'Marge nette', 'RN/ACTIF', 'IS/CA',
    'Ratio des immobilisations corporelles', 'DSO_DPO', 'Score_Métier', 'Score_ML'
]

X_anomaly = feature_store[features_anomaly].copy()

# Handle missing values and inf for anomaly detection
for col in X_anomaly.columns:
    X_anomaly[col] = X_anomaly[col].replace([np.inf, -np.inf], np.nan)
    X_anomaly[col] = X_anomaly[col].fillna(X_anomaly[col].median())

# Standardize features
scaler_anomaly = StandardScaler()
X_anomaly_scaled = scaler_anomaly.fit_transform(X_anomaly)

# 3.1 ISOLATION FOREST
print(f"\n🌲 3.1 ISOLATION FOREST")
print("-" * 25)

iso_forest = IsolationForest(
    contamination=0.1,  # 10% expected anomalies
    random_state=42,
    n_estimators=100
)

iso_predictions = iso_forest.fit_predict(X_anomaly_scaled)
iso_scores = iso_forest.decision_function(X_anomaly_scaled)

feature_store['Iso_Forest_Anomaly'] = (iso_predictions == -1).astype(int)
feature_store['Iso_Forest_Score'] = iso_scores

nb_anomalies_iso = feature_store['Iso_Forest_Anomaly'].sum()
print(f"✅ Anomalies detected (Isolation Forest): {nb_anomalies_iso} ({nb_anomalies_iso/len(feature_store)*100:.1f}%)")


# 3.2 LOCAL OUTLIER FACTOR (LOF)
print(f"\n📊 3.2 LOCAL OUTLIER FACTOR (LOF)")
print("-" * 35)

lof = LocalOutlierFactor(
    n_neighbors=20,
    contamination=0.1,
    novelty=False
)

lof_predictions = lof.fit_predict(X_anomaly_scaled)
lof_scores = lof.negative_outlier_factor_

feature_store['LOF_Anomaly'] = (lof_predictions == -1).astype(int)
feature_store['LOF_Score'] = lof_scores

nb_anomalies_lof = feature_store['LOF_Anomaly'].sum()
print(f"✅ Anomalies detected (LOF): {nb_anomalies_lof} ({nb_anomalies_lof/len(feature_store)*100:.1f}%)")

# 3.3 DBSCAN (CLUSTERING)
print(f"\n🔗 3.3 DBSCAN CLUSTERING")
print("-" * 25)

dbscan = DBSCAN(
    eps=0.5,        # Epsilon distance
    min_samples=10  # Minimum samples for a cluster
)

dbscan_labels = dbscan.fit_predict(X_anomaly_scaled)

feature_store['DBSCAN_Anomaly'] = (dbscan_labels == -1).astype(int)

nb_anomalies_dbscan = feature_store['DBSCAN_Anomaly'].sum()
nb_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)

print(f"✅ Anomalies detected (DBSCAN): {nb_anomalies_dbscan} ({nb_anomalies_dbscan/len(feature_store)*100:.1f}%)")
print(f"✅ Number of clusters: {nb_clusters}")


# --- End of re-run anomaly detection models ---


# Creation of a consolidated anomaly score
# Normalization of scores for combination

# Normalize Isolation Forest Score (0-1, 1 = more anomalous)
iso_scores_norm = (feature_store['Iso_Forest_Score'] - feature_store['Iso_Forest_Score'].min()) / \
                  (feature_store['Iso_Forest_Score'].max() - feature_store['Iso_Forest_Score'].min())
iso_scores_norm = 1 - iso_scores_norm  # Invert as smaller is more anomalous

# Normalize LOF Score (0-1, 1 = more anomalous)
lof_scores_norm = (feature_store['LOF_Score'] - feature_store['LOF_Score'].min()) / \
                  (feature_store['LOF_Score'].max() - feature_store['LOF_Score'].min())
lof_scores_norm = 1 - lof_scores_norm  # Invert as smaller is more anomalous

# DBSCAN Score (binary: 0 or 1)
dbscan_scores_norm = feature_store['DBSCAN_Anomaly'].astype(float)

# CONSOLIDATED INDEX (weighted average)
poids_iso = 0.4
poids_lof = 0.4
poids_dbscan = 0.2

feature_store['Indice_Anomalie_Consolidé'] = (
    iso_scores_norm * poids_iso +
    lof_scores_norm * poids_lof +
    dbscan_scores_norm * poids_dbscan
)

print(f"⚖️  CONSOLIDATED WEIGHTING:")
print(f"   • Isolation Forest: {poids_iso*100}%")
print(f"   • LOF: {poids_lof*100}%")
print(f"   • DBSCAN: {poids_dbscan*100}%")

# Threshold for classifying consolidated anomalies
seuil_anomalie = feature_store['Indice_Anomalie_Consolidé'].quantile(0.9)  # Top 10%
feature_store['Anomalie_Consolidée'] = (
    feature_store['Indice_Anomalie_Consolidé'] > seuil_anomalie
).astype(int)

nb_anomalies_consolidées = feature_store['Anomalie_Consolidée'].sum()

print(f"\n🎯 CONSOLIDATED RESULTS:")
print(f"   • Anomaly threshold: {seuil_anomalie:.3f}")
print(f"   • Consolidated anomalies: {nb_anomalies_consolidées} ({nb_anomalies_consolidées/len(feature_store)*100:.1f}%)")
print(f"   • Mean score: {feature_store['Indice_Anomalie_Consolidé'].mean():.3f}")
print(f"   • Max score: {feature_store['Indice_Anomalie_Consolidé'].max():.3f}")

# Agreement matrix between methods
print(f"\n📊 AGREEMENT BETWEEN METHODS:")
agreement_matrix = pd.crosstab(
    [feature_store['Iso_Forest_Anomaly'], feature_store['LOF_Anomaly']],
    feature_store['DBSCAN_Anomaly'],
    margins=True,
    margins_name="Total"
)
print("Iso_Forest x LOF vs DBSCAN:")
print(agreement_matrix)

# Overlap analysis
print(f"\n🔗 OVERLAP ANALYSIS:")
# Anomalies detected by all 3 methods
overlap_3 = ((feature_store['Iso_Forest_Anomaly'] == 1) &
             (feature_store['LOF_Anomaly'] == 1) &
             (feature_store['DBSCAN_Anomaly'] == 1)).sum()

# Anomalies detected by at least 2 methods
overlap_2plus = ((feature_store['Iso_Forest_Anomaly'] +
                  feature_store['LOF_Anomaly'] +
                  feature_store['DBSCAN_Anomaly']) >= 2).sum()

print(f"   • Detected by all 3 methods: {overlap_3}")
print(f"   • Detected by ≥2 methods: {overlap_2plus}")
print(f"   • Detected by only 1 method: {nb_anomalies_iso + nb_anomalies_lof + nb_anomalies_dbscan - 2*overlap_2plus + overlap_3}")

# Top 10 companies with highest consolidated anomaly index
print(f"\n🚨 TOP 10 MOST ANOMALOUS COMPANIES:")
top_anomalies = feature_store.nlargest(10, 'Indice_Anomalie_Consolidé')[
    ['Nom', 'Activité', 'Année', 'Indice_Anomalie_Consolidé', 'Score_Hybride', 'Catégorie_Risque']
]

for i, row in top_anomalies.iterrows():
    print(f"   {row['Nom'][:30]}... | {row['Activité']} | {row['Année']} | "
          f"Anomaly: {row['Indice_Anomalie_Consolidé']:.3f} | "
          f"Risk: {row['Score_Hybride']:.2f} ({row['Catégorie_Risque']})")

print(f"\n✅ ÉTAPE 3 TERMINATED - ANOMALY DETECTION COMPLETED")


🔀 3.4 INDICE D'ANOMALIE CONSOLIDÉ
-----------------------------------

🌲 3.1 ISOLATION FOREST
-------------------------
✅ Anomalies detected (Isolation Forest): 426 (10.0%)

📊 3.2 LOCAL OUTLIER FACTOR (LOF)
-----------------------------------
✅ Anomalies detected (LOF): 426 (10.0%)

🔗 3.3 DBSCAN CLUSTERING
-------------------------
✅ Anomalies detected (DBSCAN): 1283 (30.2%)
✅ Number of clusters: 14
⚖️  CONSOLIDATED WEIGHTING:
   • Isolation Forest: 40.0%
   • LOF: 40.0%
   • DBSCAN: 20.0%

🎯 CONSOLIDATED RESULTS:
   • Anomaly threshold: 0.434
   • Consolidated anomalies: 426 (10.0%)
   • Mean score: 0.172
   • Max score: 0.874

📊 AGREEMENT BETWEEN METHODS:
Iso_Forest x LOF vs DBSCAN:
DBSCAN_Anomaly                     0     1  Total
Iso_Forest_Anomaly LOF_Anomaly                   
0                  0            2853   674   3527
                   1              58   243    301
1                  0              60   241    301
                   1               0   125    125
Total

In [26]:
# SYNTHÈSE FINALE ET EXPORT DES RÉSULTATS
print("\n📊 SYNTHÈSE FINALE - CARTE DE RISQUE IA")
print("=" * 50)

# Création du dataset final avec tous les scores
colonnes_finales = [
    'Nom', 'Activité', 'Année',
    # 'Liquidité générale', 'Marge nette', 'RN/ACTIF', 'IS/CA',
    'Ratio des immobilisations corporelles', 'DSO_DPO',
    'Score_Métier', 'Score_ML', 'Score_Hybride', 'Catégorie_Risque',
    'Indice_Anomalie_Consolidé', 'Anomalie_Consolidée',
    'Iso_Forest_Anomaly', 'LOF_Anomaly', 'DBSCAN_Anomaly'
]

carte_risque_finale = feature_store[colonnes_finales].copy()

# Statistiques finales
print(f"📈 RÉSULTATS FINAUX:")
print(f"   • Total entreprises analysées: {carte_risque_finale['Nom'].nunique()}")
print(f"   • Période d'analyse: {carte_risque_finale['Année'].min()}-{carte_risque_finale['Année'].max()}")
print(f"   • Observations traitées: {len(carte_risque_finale)}")

print(f"\n🎯 DISTRIBUTION SCORES HYBRIDES:")
risk_stats = carte_risque_finale['Catégorie_Risque'].value_counts(normalize=True) * 100
for category, percentage in risk_stats.items():
    print(f"   • {category}: {percentage:.1f}%")

print(f"\n🚨 DÉTECTION D'ANOMALIES:")
print(f"   • Anomalies consolidées: {carte_risque_finale['Anomalie_Consolidée'].sum()} ({carte_risque_finale['Anomalie_Consolidée'].mean()*100:.1f}%)")
print(f"   • Score d'anomalie moyen: {carte_risque_finale['Indice_Anomalie_Consolidé'].mean():.3f}")

# Analyse sectorielle finale
print(f"\n🏭 ANALYSE SECTORIELLE FINALE:")
analyse_secteur = carte_risque_finale.groupby('Activité').agg({
    'Score_Hybride': 'mean',
    'Indice_Anomalie_Consolidé': 'mean',
    'Anomalie_Consolidée': 'sum',
    'Nom': 'nunique'
}).round(3)

analyse_secteur.columns = ['Score_Hybride_Moyen', 'Anomalie_Score_Moyen',
                          'Nb_Anomalies', 'Nb_Entreprises']
analyse_secteur['Taux_Anomalie_%'] = (analyse_secteur['Nb_Anomalies'] / analyse_secteur['Nb_Entreprises'] * 100).round(1)

print(analyse_secteur)

# Export des résultats
carte_risque_finale.to_csv('Carte_Risque_IA_Finale.csv', index=False)
print(f"\n💾 EXPORT RÉALISÉ:")
print(f"   ✅ Carte_Risque_IA_Finale.csv")
print(f"   📊 {len(colonnes_finales)} colonnes exportées")
print(f"   🔍 Prêt pour dashboard et analyses avancées")

# Métriques de qualité
print(f"\n🎖️  MÉTRIQUES DE QUALITÉ:")
print(f"   • Model ML Accuracy: {rf_accuracy:.3f}")
print(f"   • Model ML AUC-ROC: {rf_auc:.3f}")
print(f"   • Cohérence anomalies (3 méthodes): {overlap_3} entreprises")
print(f"   • Couverture complète: 100% des données")

print(f"\n✅ ÉTAPES 2 & 3 COMPLÈTEMENT TERMINÉES")


📊 SYNTHÈSE FINALE - CARTE DE RISQUE IA
📈 RÉSULTATS FINAUX:
   • Total entreprises analysées: 885
   • Période d'analyse: 2020-2024
   • Observations traitées: 4254

🎯 DISTRIBUTION SCORES HYBRIDES:
   • Haut: 66.6%
   • Modéré: 33.4%

🚨 DÉTECTION D'ANOMALIES:
   • Anomalies consolidées: 426 (10.0%)
   • Score d'anomalie moyen: 0.172

🏭 ANALYSE SECTORIELLE FINALE:
            Score_Hybride_Moyen  Anomalie_Score_Moyen  Nb_Anomalies  \
Activité                                                              
Activité 1                4.724                 0.210            39   
Activité 2                4.855                 0.225           123   
Activité 3                4.444                 0.198             2   
Activité 4                5.305                 0.266           259   
Activité 5                4.007                 0.087             0   
Activité 6                3.782                 0.099             3   

            Nb_Entreprises  Taux_Anomalie_%  
Activité           

In [27]:
print("🎯 ÉTAPE 4: VISUALISATION & DASHBOARD STREAMLIT")
print("=" * 50)

# Installation des packages nécessaires
import sys
import subprocess

packages_to_install = ['streamlit', 'plotly', 'seaborn', 'shap']

for package in packages_to_install:
    try:
        __import__(package)
        print(f"✅ {package} déjà installé")
    except ImportError:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ {package} installé avec succès")
        except Exception as e:
            print(f"⚠️ Erreur installation {package}: {e}")

print("\n📦 VÉRIFICATION DES IMPORTS:")
import pandas as pd
import numpy as np
import streamlit
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
print("✅ Tous les imports principaux sont disponibles")

try:
    import shap
    print("✅ SHAP disponible pour l'explicabilité")
    shap_available = True
except ImportError:
    print("⚠️ SHAP non disponible, mode explicabilité limité")
    shap_available = False

🎯 ÉTAPE 4: VISUALISATION & DASHBOARD STREAMLIT
✅ streamlit installé avec succès
✅ plotly déjà installé
✅ seaborn déjà installé
✅ shap déjà installé

📦 VÉRIFICATION DES IMPORTS:
✅ Tous les imports principaux sont disponibles
✅ SHAP disponible pour l'explicabilité


In [28]:
# Création de l'application Streamlit sans l'importer directement
print("🎯 ÉTAPE 4: CRÉATION DU DASHBOARD STREAMLIT")
print("=" * 50)

# Chargement des données nécessaires pour le dashboard
try:
    # Essayer de charger les données finales
    carte_risque = pd.read_csv('Carte_Risque_IA_Finale.csv')
    print(f"✅ Données chargées: {carte_risque.shape}")
except:
    # Recréer rapidement les données si nécessaire
    print("⚠️ Recréation des données pour le dashboard...")
    df_raw = pd.read_excel('Data_09092025.xlsx')

    # Colonnes de base
    carte_risque = df_raw[['Nom', 'Activité', 'Année', 'Liquidité générale', 'Marge nette',
                          'IS/CA', 'RN/ACTIF', 'Délai moyen de règlement des clients',
                          'Délai moyen de règlement des fournisseurs',
                          'Ratio des immobilisations corporelles']].copy()

    carte_risque.columns = ['Nom', 'Activité', 'Année', 'Liquidité_générale', 'Marge_nette',
                           'IS_CA', 'RN_ACTIF', 'Délai_clients', 'Délai_fournisseurs',
                           'Ratio_immobilisations']

    # Nettoyage des données
    for col in ['Liquidité_générale', 'Marge_nette', 'IS_CA', 'RN_ACTIF', 'Ratio_immobilisations']:
        carte_risque[col].fillna(carte_risque[col].median(), inplace=True)

    # Calculs de base pour le dashboard
    carte_risque['DSO_DPO'] = carte_risque['Délai_clients'] - carte_risque['Délai_fournisseurs']

    # Score hybride simplifié
    carte_risque['Score_Hybride'] = (
        (carte_risque['Liquidité_générale'] / carte_risque['Liquidité_générale'].max()) * 2.5 +
        ((carte_risque['RN_ACTIF'] + 0.1) / (carte_risque['RN_ACTIF'].max() + 0.1)) * 2.5 +
        (1 / (carte_risque['DSO_DPO'].abs() + 1)) * 2 +
        (1 / (carte_risque['IS_CA'] + 0.01)) * 1.5 * 0.1 +
        (1 / (abs(carte_risque['Ratio_immobilisations'] - 0.1) + 0.01)) * 1.5 * 0.1
    )

    # Normalisation du score sur 10
    carte_risque['Score_Hybride'] = ((carte_risque['Score_Hybride'] - carte_risque['Score_Hybride'].min()) /
                                    (carte_risque['Score_Hybride'].max() - carte_risque['Score_Hybride'].min()) * 10)

    # Catégories de risque
    def classify_risk(score):
        if score <= 3:
            return "Très Haut"
        elif score <= 5:
            return "Haut"
        elif score <= 7:
            return "Modéré"
        else:
            return "Faible"

    carte_risque['Catégorie_Risque'] = carte_risque['Score_Hybride'].apply(classify_risk)

    # Anomalies simplifiées (basées sur les quartiles)
    seuil_anomalie = carte_risque['Score_Hybride'].quantile(0.1)  # 10% les plus bas
    carte_risque['Anomalie_Consolidée'] = (carte_risque['Score_Hybride'] <= seuil_anomalie).astype(int)

print(f"📊 Données préparées pour le dashboard: {carte_risque.shape}")
print(f"Colonnes disponibles: {list(carte_risque.columns)}")

# Vérification des données
print(f"\n📈 APERÇU DES DONNÉES POUR LE DASHBOARD:")
print(f"   • Entreprises: {carte_risque['Nom'].nunique()}")
print(f"   • Secteurs: {carte_risque['Activité'].nunique()}")
print(f"   • Années: {carte_risque['Année'].min()}-{carte_risque['Année'].max()}")
print(f"   • Score moyen: {carte_risque['Score_Hybride'].mean():.2f}")
print(f"   • Anomalies: {carte_risque['Anomalie_Consolidée'].sum()}")

# Distribution des catégories de risque
print(f"\n🎯 DISTRIBUTION DES RISQUES:")
risk_dist = carte_risque['Catégorie_Risque'].value_counts()
for category, count in risk_dist.items():
    print(f"   • {category}: {count} ({count/len(carte_risque)*100:.1f}%)")

🎯 ÉTAPE 4: CRÉATION DU DASHBOARD STREAMLIT
✅ Données chargées: (4254, 14)
📊 Données préparées pour le dashboard: (4254, 14)
Colonnes disponibles: ['Nom', 'Activité', 'Année', 'Ratio des immobilisations corporelles', 'DSO_DPO', 'Score_Métier', 'Score_ML', 'Score_Hybride', 'Catégorie_Risque', 'Indice_Anomalie_Consolidé', 'Anomalie_Consolidée', 'Iso_Forest_Anomaly', 'LOF_Anomaly', 'DBSCAN_Anomaly']

📈 APERÇU DES DONNÉES POUR LE DASHBOARD:
   • Entreprises: 885
   • Secteurs: 6
   • Années: 2020-2024
   • Score moyen: 4.53
   • Anomalies: 426

🎯 DISTRIBUTION DES RISQUES:
   • Haut: 2834 (66.6%)
   • Modéré: 1420 (33.4%)


In [29]:
# Correction du calcul et création du code Streamlit complet
print("🔧 CORRECTION ET CRÉATION DU CODE STREAMLIT")
print("=" * 50)

# Recalcul correct des scores
import pandas as pd
import numpy as np

# Rechargement propre des données
df_raw = pd.read_excel('Data_09092025.xlsx')

# Création du dataset pour le dashboard
dashboard_data = df_raw[['Nom', 'Activité', 'Année', 'Liquidité générale', 'Marge nette',
                        'IS/CA', 'RN/ACTIF', 'Délai moyen de règlement des clients',
                        'Délai moyen de règlement des fournisseurs',
                        'Ratio des immobilisations corporelles']].copy()

dashboard_data.columns = ['Nom', 'Activité', 'Année', 'Liquidité_générale', 'Marge_nette',
                         'IS_CA', 'RN_ACTIF', 'Délai_clients', 'Délai_fournisseurs',
                         'Ratio_immobilisations']

# Nettoyage des données
numeric_cols = ['Liquidité_générale', 'Marge_nette', 'IS_CA', 'RN_ACTIF', 'Ratio_immobilisations']
for col in numeric_cols:
    dashboard_data[col] = dashboard_data[col].fillna(dashboard_data[col].median())

# Calcul du DSO-DPO
dashboard_data['DSO_DPO'] = dashboard_data['Délai_clients'] - dashboard_data['Délai_fournisseurs']

# Score hybride amélioré
dashboard_data['Score_Liquidité'] = np.clip(dashboard_data['Liquidité_générale'], 0, 10)
dashboard_data['Score_Rentabilité'] = np.clip((dashboard_data['RN_ACTIF'] + 0.1) * 50, 0, 10)
dashboard_data['Score_BFR'] = np.clip(10 - (dashboard_data['DSO_DPO'].abs() / 100), 0, 10)
dashboard_data['Score_Fiscal'] = np.clip(10 - (dashboard_data['IS_CA'] * 100), 0, 10)
dashboard_data['Score_Structure'] = np.clip(10 - abs(dashboard_data['Ratio_immobilisations'] - 0.1) * 50, 0, 10)

# Score hybride final
dashboard_data['Score_Hybride'] = (
    dashboard_data['Score_Liquidité'] * 0.25 +
    dashboard_data['Score_Rentabilité'] * 0.25 +
    dashboard_data['Score_BFR'] * 0.20 +
    dashboard_data['Score_Fiscal'] * 0.15 +
    dashboard_data['Score_Structure'] * 0.15
)

# Catégorie de risque
def classify_risk(score):
    if score <= 3:
        return "Très Haut"
    elif score <= 5:
        return "Haut"
    elif score <= 7:
        return "Modéré"
    else:
        return "Faible"

dashboard_data['Catégorie_Risque'] = dashboard_data['Score_Hybride'].apply(classify_risk)

# Anomalies (seuil à 10% les plus bas)
seuil_anomalie = dashboard_data['Score_Hybride'].quantile(0.1)
dashboard_data['Anomalie'] = dashboard_data['Score_Hybride'] <= seuil_anomalie

print(f"✅ Données correctement préparées: {dashboard_data.shape}")
print(f"📊 Score moyen: {dashboard_data['Score_Hybride'].mean():.2f}")
print(f"🚨 Anomalies: {dashboard_data['Anomalie'].sum()} ({dashboard_data['Anomalie'].sum()/len(dashboard_data)*100:.1f}%)")

# Distribution des risques
print(f"\n🎯 DISTRIBUTION DES CATÉGORIES DE RISQUE:")
risk_dist = dashboard_data['Catégorie_Risque'].value_counts()
for category, count in risk_dist.items():
    print(f"   • {category}: {count} ({count/len(dashboard_data)*100:.1f}%)")

# Sauvegarde pour le dashboard
dashboard_data.to_csv('Dashboard_Data.csv', index=False)
print(f"\n💾 Données sauvegardées: Dashboard_Data.csv")

🔧 CORRECTION ET CRÉATION DU CODE STREAMLIT
✅ Données correctement préparées: (4254, 19)
📊 Score moyen: 5.80
🚨 Anomalies: 426 (10.0%)

🎯 DISTRIBUTION DES CATÉGORIES DE RISQUE:
   • Modéré: 2980 (70.1%)
   • Haut: 785 (18.5%)
   • Faible: 400 (9.4%)
   • Très Haut: 89 (2.1%)

💾 Données sauvegardées: Dashboard_Data.csv


In [30]:
# 🎯 DASHBOARD CARTE DE RISQUE IA - STREAMLIT
# Étape 4: Visualisation & Dashboard Interactif

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configuration de la page
st.set_page_config(
    page_title="Carte de Risque IA",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# CSS personnalisé pour améliorer l'apparence
st.markdown("""
<style>
    .metric-card {
        background-color: #f0f2f6;
        padding: 1rem;
        border-radius: 0.5rem;
        border-left: 5px solid #1f77b4;
    }
    .risk-high { color: #d62728; font-weight: bold; }
    .risk-moderate { color: #ff7f0e; font-weight: bold; }
    .risk-low { color: #2ca02c; font-weight: bold; }
    .risk-very-high { color: #8b0000; font-weight: bold; }
</style>
""", unsafe_allow_html=True)

@st.cache_data
def load_data():
    """Charge les données du dashboard"""
    try:
        data = pd.read_csv('Dashboard_Data.csv')
        return data
    except FileNotFoundError:
        st.error("Fichier Dashboard_Data.csv non trouvé. Veuillez exécuter l'étape de préparation des données.")
        return None

def color_risk_category(category):
    """Retourne la couleur selon la catégorie de risque"""
    colors = {
        'Très Haut': '#8b0000',
        'Haut': '#d62728',
        'Modéré': '#ff7f0e',
        'Faible': '#2ca02c'
    }
    return colors.get(category, '#1f77b4')

def main():
    # Titre principal
    st.title("🎯 Carte de Risque IA - Dashboard Interactif")
    st.markdown("---")

    # Chargement des données
    data = load_data()
    if data is None:
        return

    # SIDEBAR - Filtres
    st.sidebar.header("🔧 Filtres")

    # Filtre par année
    years = sorted(data['Année'].unique())
    selected_years = st.sidebar.multiselect(
        "📅 Sélectionner les années",
        years,
        default=years
    )

    # Filtre par activité (secteur)
    activities = sorted(data['Activité'].unique())
    selected_activities = st.sidebar.multiselect(
        "🏭 Sélectionner les secteurs",
        activities,
        default=activities
    )

    # Filtre par catégorie de risque
    risk_categories = sorted(data['Catégorie_Risque'].unique())
    selected_risks = st.sidebar.multiselect(
        "⚠️ Catégories de risque",
        risk_categories,
        default=risk_categories
    )

    # Application des filtres
    filtered_data = data[
        (data['Année'].isin(selected_years)) &
        (data['Activité'].isin(selected_activities)) &
        (data['Catégorie_Risque'].isin(selected_risks))
    ]

    # Métriques principales
    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.metric(
            "👥 Entreprises",
            f"{filtered_data['Nom'].nunique():,}",
            delta=f"{len(filtered_data):,} observations"
        )

    with col2:
        avg_score = filtered_data['Score_Hybride'].mean()
        st.metric(
            "📊 Score Moyen",
            f"{avg_score:.2f}/10",
            delta=f"±{filtered_data['Score_Hybride'].std():.2f}"
        )

    with col3:
        anomalies = filtered_data['Anomalie'].sum()
        anomaly_rate = (anomalies / len(filtered_data)) * 100
        st.metric(
            "🚨 Anomalies",
            f"{anomalies:,}",
            delta=f"{anomaly_rate:.1f}%"
        )

    with col4:
        high_risk = len(filtered_data[filtered_data['Catégorie_Risque'].isin(['Très Haut', 'Haut'])])
        high_risk_rate = (high_risk / len(filtered_data)) * 100
        st.metric(
            "⚠️ Risque Élevé",
            f"{high_risk:,}",
            delta=f"{high_risk_rate:.1f}%"
        )

    st.markdown("---")

    # TABLEAU PRINCIPAL DES ENTREPRISES
    st.header("📋 Tableau des Entreprises")

    # Options d'affichage
    col1, col2 = st.columns([3, 1])
    with col1:
        show_anomalies_only = st.checkbox("🚨 Afficher uniquement les anomalies")
    with col2:
        max_rows = st.selectbox("Nombre de lignes", [50, 100, 200, 500], index=1)

    # Filtrage pour anomalies
    table_data = filtered_data.copy()
    if show_anomalies_only:
        table_data = table_data[table_data['Anomalie'] == True]

    # Préparation du tableau
    display_columns = [
        'Nom', 'Activité', 'Année', 'Score_Hybride',
        'Catégorie_Risque', 'Anomalie', 'Liquidité_générale',
        'RN_ACTIF', 'Marge_nette', 'DSO_DPO'
    ]

    table_display = table_data[display_columns].head(max_rows).copy()
    table_display['Score_Hybride'] = table_display['Score_Hybride'].round(2)
    table_display['Anomalie'] = table_display['Anomalie'].map({True: "🚨 Oui", False: "✅ Non"})

    # Affichage du tableau avec style
    st.dataframe(
        table_display.style.applymap(
            lambda x: f'color: {color_risk_category(x)}' if x in ['Très Haut', 'Haut', 'Modéré', 'Faible'] else '',
            subset=['Catégorie_Risque']
        ),
        height=400,
        use_container_width=True
    )

    st.markdown("---")

    # GRAPHIQUES
    st.header("📈 Visualisations")

    # Onglets pour les différents graphiques
    tab1, tab2, tab3, tab4 = st.tabs(["🔥 Heatmap", "📊 Scatter Plot", "📉 Histogrammes", "🔍 Explicabilité"])

    with tab1:
        st.subheader("🔥 Heatmap des Scores par Secteur et Année")

        # Création de la heatmap
        heatmap_data = filtered_data.groupby(['Activité', 'Année'])['Score_Hybride'].mean().unstack()

        fig_heatmap = px.imshow(
            heatmap_data.values,
            x=heatmap_data.columns,
            y=heatmap_data.index,
            color_continuous_scale='RdYlGn',
            aspect='auto',
            title="Score Hybride Moyen par Secteur et Année"
        )
        fig_heatmap.update_layout(height=500)
        st.plotly_chart(fig_heatmap, use_container_width=True)

    with tab2:
        st.subheader("📊 Scatter Plot: RN/ACTIF vs Marge Nette")

        fig_scatter = px.scatter(
            filtered_data.sample(min(1000, len(filtered_data))),  # Échantillon pour performance
            x='RN_ACTIF',
            y='Marge_nette',
            color='Catégorie_Risque',
            size='Score_Hybride',
            hover_data=['Nom', 'Activité', 'Année'],
            title="Relation entre Rentabilité des Actifs et Marge Nette",
            color_discrete_map={
                'Très Haut': '#8b0000',
                'Haut': '#d62728',
                'Modéré': '#ff7f0e',
                'Faible': '#2ca02c'
            }
        )
        fig_scatter.update_layout(height=500)
        st.plotly_chart(fig_scatter, use_container_width=True)

    with tab3:
        st.subheader("📉 Distribution DSO-DPO")

        col1, col2 = st.columns(2)

        with col1:
            # Histogramme DSO-DPO
            fig_hist = px.histogram(
                filtered_data,
                x='DSO_DPO',
                nbins=50,
                title="Distribution DSO-DPO (Besoin en Fonds de Roulement)",
                color_discrete_sequence=['#1f77b4']
            )
            fig_hist.update_layout(height=400)
            st.plotly_chart(fig_hist, use_container_width=True)

        with col2:
            # Box plot DSO-DPO par secteur
            fig_box = px.box(
                filtered_data,
                x='Activité',
                y='DSO_DPO',
                title="DSO-DPO par Secteur d'Activité"
            )
            fig_box.update_layout(height=400)
            st.plotly_chart(fig_box, use_container_width=True)

    with tab4:
        st.subheader("🔍 Mode Explicabilité")

        st.info("💡 Cette section explique les facteurs influençant les scores de risque")

        # Importance des features (simulation)
        features_importance = {
            'Liquidité Générale': 0.25,
            'Rentabilité (RN/ACTIF)': 0.25,
            'BFR (DSO-DPO)': 0.20,
            'Fiscalité (IS/CA)': 0.15,
            'Structure (Immobilisations)': 0.15
        }

        # Graphique d'importance
        fig_importance = px.bar(
            x=list(features_importance.values()),
            y=list(features_importance.keys()),
            orientation='h',
            title="Importance des Facteurs dans le Score Hybride",
            color=list(features_importance.values()),
            color_continuous_scale='viridis'
        )
        fig_importance.update_layout(height=400)
        st.plotly_chart(fig_importance, use_container_width=True)

        # Explication textuelle
        st.markdown("""
        **📋 Interprétation des Facteurs:**
        - **Liquidité Générale (25%)**: Capacité à honorer les dettes à court terme
        - **Rentabilité RN/ACTIF (25%)**: Efficacité dans l'utilisation des actifs
        - **BFR DSO-DPO (20%)**: Gestion du cycle d'exploitation
        - **Fiscalité IS/CA (15%)**: Optimisation fiscale
        - **Structure Immobilisations (15%)**: Équilibre de la structure d'actifs
        """)

    st.markdown("---")

    # ANALYSE SECTORIELLE
    st.header("🏭 Analyse Sectorielle")

    sector_analysis = filtered_data.groupby('Activité').agg({
        'Score_Hybride': ['mean', 'std', 'count'],
        'Anomalie': 'sum',
        'Nom': 'nunique'
    }).round(2)

    sector_analysis.columns = ['Score_Moyen', 'Écart_Type', 'Nb_Observations', 'Nb_Anomalies', 'Nb_Entreprises']
    sector_analysis['Taux_Anomalie_%'] = (sector_analysis['Nb_Anomalies'] / sector_analysis['Nb_Observations'] * 100).round(1)

    st.dataframe(sector_analysis, use_container_width=True)

    # Footer
    st.markdown("---")
    st.markdown("**🎯 Carte de Risque IA** - Dashboard développé pour l'analyse financière avancée")

if __name__ == "__main__":
    main()

2025-10-31 23:09:44.599 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 23:09:44.600 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 23:09:44.986 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-10-31 23:09:44.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 23:09:44.989 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 23:09:44.990 No runtime found, using MemoryCacheStorageManager
2025-10-31 23:09:44.996 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 23:09:44.997 Thread 'Ma

In [34]:
!pip install streamlit pyngrok


In [ ]:
#Etape de création dashbord Streamlit

In [35]:
%%writefile dashboard.py
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


st.set_page_config(page_title="Carte de Risque IA", layout="wide")

# Charger les données préalablement uploadées
@st.cache_data
def load_data():
    """Charge les données du dashboard"""
    try:
        data = pd.read_csv('Dashboard_Data.csv')
        return data
    except FileNotFoundError:
        st.error("Fichier Dashboard_Data.csv non trouvé. Veuillez exécuter l'étape de préparation des données.")
        return None

df = load_data()

if df is None:
    st.stop()

# Charger le modèle et le scaler (assurez-vous qu'ils sont sauvegardés ou recréés si nécessaire)
# Pour cet exemple, nous allons recréer un scaler et un modèle simples pour la démo SHAP
@st.cache_resource
def load_model_and_scaler(data):
    features = [
        'Liquidité_générale','Marge_nette','RN_ACTIF','IS_CA',
        'Ratio_immobilisations','DSO_DPO','Délai_clients','Délai_fournisseurs'
    ]
    X = data[features].copy()

    # Handle missing values and inf for anomaly detection
    for col in X.columns:
        X[col] = X[col].replace([np.inf, -np.inf], np.nan)
        X[col] = X[col].fillna(X[col].median())

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Créer des labels de risque simples pour l'exemple SHAP
    # Use the 'Score_Hybride' calculated in the dashboard data as the target
    threshold_risque = data['Score_Hybride'].quantile(0.3)
    y = (data['Score_Hybride'] <= threshold_risque).astype(int)


    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_scaled, y)

    return model, scaler, X_scaled, features

model, scaler, X_scaled, features_for_shap = load_model_and_scaler(df)

# Sidebar
years = sorted(df["Année"].unique())
selections = st.sidebar.multiselect("Années", years, default=years)
activities = sorted(df["Activité"].unique())
acts = st.sidebar.multiselect("Secteurs", activities, default=activities)

# Filtrer
data = df[df["Année"].isin(selections) & df["Activité"].isin(acts)].copy() # Use .copy() to avoid SettingWithCopyWarning

# Titre
st.title("📊 Carte de Risque IA")

# KPI
col1, col2, col3 = st.columns(3)
col1.metric("Entreprises", data["Nom"].nunique())
col2.metric("Score moyen", f"{data['Score_Hybride'].mean():.2f}/10")
col3.metric("Anomalies", int(data["Anomalie"].sum()))

st.markdown("---")

# Tableau
st.header("Tableau des entreprises")
table = data[["Nom","Activité","Année","Score_Hybride","Catégorie_Risque","Anomalie"]].copy() # Use .copy()
table["Anomalie"] = table["Anomalie"].map({1:"Oui",0:"Non"})
st.dataframe(table, height=400, use_container_width=True)

st.markdown("---")

# GRAPHIQUES
st.header("📈 Visualisations")

# Onglets pour les différents graphiques
tab1, tab2, tab3, tab4 = st.tabs(["🔥 Heatmap", "📊 Scatter Plot", "📉 Histogrammes", "🔍 Explicabilité SHAP"])

with tab1:
    st.subheader("🔥 Heatmap des Scores par Secteur et Année")
    if not data.empty:
        heat = data.groupby(["Activité","Année"])["Score_Hybride"].mean().unstack()
        fig1 = px.imshow(heat, color_continuous_scale="RdYlGn", aspect="auto", title="Score Hybride Moyen par Secteur et Année")
        st.plotly_chart(fig1, use_container_width=True)
    else:
        st.warning("Pas de données disponibles pour la Heatmap avec les filtres actuels.")


with tab2:
    st.subheader("📊 Scatter Plot: RN/ACTIF vs Marge nette")
    if not data.empty:
        # Assurez-vous que les colonnes existent dans les données filtrées
        scatter_data = data.dropna(subset=['RN_ACTIF', 'Marge_nette'])
        if not scatter_data.empty:
            fig2 = px.scatter(scatter_data, x="RN_ACTIF", y="Marge_nette",
                              color="Catégorie_Risque", size="Score_Hybride",
                              hover_data=["Nom","Année","Activité"],
                              title="Relation entre Rentabilité des Actifs et Marge Nette",
                              color_discrete_map={
                                'Très Haut': '#8b0000',
                                'Haut': '#d62728',
                                'Modéré': '#ff7f0e',
                                'Faible': '#2ca02c'
                                }
                            )
            st.plotly_chart(fig2, use_container_width=True)
        else:
            st.warning("Pas de données disponibles pour le Scatter Plot avec les filtres actuels (vérifiez RN/ACTIF et Marge_nette).")
    else:
         st.warning("Pas de données disponibles pour le Scatter Plot avec les filtres actuels.")


with tab3:
    st.subheader("📉 Distribution DSO-DPO")
    if not data.empty:
        col_hist, col_box = st.columns(2)
        with col_hist:
            fig3 = px.histogram(data.dropna(subset=['DSO_DPO']), x="DSO_DPO", nbins=50, title="Distribution DSO-DPO")
            st.plotly_chart(fig3, use_container_width=True)
        with col_box:
             fig_box = px.box(data.dropna(subset=['DSO_DPO', 'Activité']), x='Activité', y='DSO_DPO', title="DSO-DPO par Secteur d'Activité")
             st.plotly_chart(fig_box, use_container_width=True)
    else:
         st.warning("Pas de données disponibles pour les Histogrammes avec les filtres actuels.")

with tab4: # This is the new tab for SHAP
    st.subheader("🔍 Explication SHAP des Prédictions")
    st.info("Sélectionnez une entreprise pour voir les facteurs qui influencent sa prédiction de risque (Haut vs Bas).")

    # Add filters within the SHAP tab
    st.markdown("##### Filtres pour l'explication SHAP")
    col1_filter, col2_filter, col3_filter = st.columns(3)

    with col1_filter:
        shap_years = sorted(data['Année'].unique())
        selected_shap_years = st.multiselect(
            "📅 Années SHAP",
            shap_years,
            default=shap_years,
            key='shap_years' # Add a unique key
        )

    with col2_filter:
        shap_activities = sorted(data['Activité'].unique())
        selected_shap_activities = st.multiselect(
            "🏭 Secteurs SHAP",
            shap_activities,
            default=shap_activities,
            key='shap_activities' # Add a unique key
        )

    with col3_filter:
        shap_risk_categories = sorted(data['Catégorie_Risque'].unique())
        selected_shap_risks = st.multiselect(
            "⚠️ Catégories de risque SHAP",
            shap_risk_categories,
            default=shap_risk_categories,
            key='shap_risks' # Add a unique key
        )

    # Apply filters to get the subset of data for SHAP selection
    filtered_shap_data = data[
        (data['Année'].isin(selected_shap_years)) &
        (data['Activité'].isin(selected_shap_activities)) &
        (data['Catégorie_Risque'].isin(selected_shap_risks))
    ]

    if not filtered_shap_data.empty:
        # Ensure the selected company is in the filtered data for SHAP
        company_names_for_shap = filtered_shap_data['Nom'].unique()
        selected_company = st.selectbox("Choisir une entreprise", company_names_for_shap, key='shap_company_select') # Add a unique key

        if selected_company:
            # Get the row for the selected company from the *filtered* data
            selected_company_row = filtered_shap_data[filtered_shap_data['Nom'] == selected_company].iloc[0]

            # Find the index of this row in the *original* dataframe used for training (df)
            # This index is needed to get the correct scaled data from X_scaled
            original_index_for_shap = df[(df['Nom'] == selected_company_row['Nom']) &
                                         (df['Année'] == selected_company_row['Année']) &
                                         (df['Activité'] == selected_company_row['Activité'])].index

            if not original_index_for_shap.empty:
                idx = original_index_for_shap[0]

                # Check if SHAP values are available for this index
                if idx < X_scaled.shape[0]:

                    try:
                        # Get the single sample from the *scaled* data using the original index
                        single_sample_scaled = X_scaled[idx, :].reshape(1, -1)

                        # Get the predicted class for this instance
                        predicted_class = model.predict(single_sample_scaled)[0]

                        if predicted_class == 0:
                            st.info(f"Le modèle prédit un risque **Bas** pour l'entreprise {selected_company_row['Nom']} en {selected_company_row['Année']}.")
                            st.write("Le graphique SHAP Force Plot ci-dessous montre les facteurs qui poussent la prédiction vers le risque **Bas**.")
                            # Calculate SHAP values for the "risk low" class (index 0)
                            explainer = shap.TreeExplainer(model)
                            shap_values = explainer.shap_values(single_sample_scaled)

                            if isinstance(shap_values, list) and len(shap_values) > 0:
                                shap_values_risk_low = shap_values[0][0] # SHAP values for the "risk low" class (index 0)
                                expected_value_risk_low = explainer.expected_value[0] # Expected value for class 0

                                # Create a SHAP force plot for the "risk low" class
                                shap.force_plot(
                                    expected_value_risk_low,
                                    shap_values_risk_low,
                                    scaler.inverse_transform(single_sample_scaled)[0],
                                    feature_names=features_for_shap,
                                    matplotlib=True,
                                    show=False
                                )
                                plt.gcf().set_size_inches(10, 5)
                                st.pyplot(plt.gcf(), bbox_inches='tight')
                            else:
                                st.warning("Impossible d'obtenir les valeurs SHAP pour la classe 'Risque Bas' pour cette entreprise.")


                        elif predicted_class == 1:
                             st.info(f"Le modèle prédit un risque **Haut** pour l'entreprise {selected_company_row['Nom']} en {selected_company_row['Année']}.")
                             st.write("Le graphique SHAP Force Plot ci-dessous montre les facteurs qui poussent la prédiction vers le risque **Haut**.")
                             # Calculate SHAP values for the "risk high" class (index 1)
                             explainer = shap.TreeExplainer(model)
                             shap_values = explainer.shap_values(single_sample_scaled)

                             if isinstance(shap_values, list) and len(shap_values) > 1:
                                shap_values_risk_high = shap_values[1][0] # SHAP values for the "risk high" class (index 1)
                                expected_value_risk_high = explainer.expected_value[1] # Expected value for class 1

                                # Create a SHAP force plot for the "risk high" class
                                shap.force_plot(
                                    expected_value_risk_high,
                                    shap_values_risk_high,
                                    scaler.inverse_transform(single_sample_scaled)[0],
                                    feature_names=features_for_shap,
                                    matplotlib=True,
                                    show=False
                                )
                                plt.gcf().set_size_inches(10, 5)
                                st.pyplot(plt.gcf(), bbox_inches='tight')
                             else:
                                st.warning("Impossible d'obtenir les valeurs SHAP pour la classe 'Risque Haut' pour cette entreprise. Le modèle pourrait prédire l'autre classe avec une très haute confiance.")

                        st.markdown("""
                        **Comment interpréter le graphique SHAP Force Plot :**
                        - Les valeurs SHAP poussent la prédiction de base (expected value) vers la prédiction finale de l'entreprise.
                        - Les flèches rouges indiquent les facteurs qui augmentent la probabilité de risque (pour la classe visualisée).
                        - Les flèches bleues indiquent les facteurs qui diminuent la probabilité de risque (pour la classe visualisée).
                        - La longueur de la flèche indique l'ampleur de l'impact du facteur.
                        """)

                    except Exception as e:
                         st.error(f"Erreur lors du calcul ou de l'affichage SHAP : {e}")
                         st.warning("Assurez-vous que le modèle et les données utilisées pour SHAP sont cohérents.")
                else:
                     st.warning("Index de l'entreprise sélectionnée hors des limites des données SHAP.")
            else:
                st.warning("Entreprise sélectionnée non trouvée dans les données d'origine pour SHAP.")
    else:
        st.warning("Pas de données disponibles pour l'explication SHAP avec les filtres actuels.")

st.markdown("---")

# ANALYSE SECTORIELLE
st.header("🏭 Analyse Sectorielle")
if not data.empty:
    sector_analysis = data.groupby('Activité').agg({
        'Score_Hybride': ['mean', 'std', 'count'],
        'Anomalie': 'sum',
        'Nom': 'nunique'
    }).round(2)

    sector_analysis.columns = ['Score_Moyen', 'Écart_Type', 'Nb_Observations', 'Nb_Anomalies', 'Nb_Entreprises']
    # Avoid division by zero
    sector_analysis['Taux_Anomalie_%'] = (sector_analysis['Nb_Anomalies'] / sector_analysis['Nb_Observations'] * 100).replace([np.inf, -np.inf], 0).fillna(0).round(1)


    st.dataframe(sector_analysis, use_container_width=True)
else:
    st.warning("Pas de données disponibles pour l'analyse sectorielle avec les filtres actuels.")


# Footer
st.markdown("---")
st.markdown("**🎯 Carte de Risque IA** - Dashboard développé pour l'analyse financière avancée")

Overwriting dashboard.py


In [ ]:
ngrok.set_auth_token("33vULSRVM1BD6ge8wkpqwZvUijw_4TSQVeqmez3uTMKwQdc9o")

In [39]:
# Lancer Streamlit en arrière-plan
import threading
from pyngrok import ngrok

# Set the ngrok auth token
ngrok.set_auth_token("33vULSRVM1BD6ge8wkpqwZvUijw_4TSQVeqmez3uTMKwQdc9o")

def run_streamlit():
    !streamlit run dashboard.py --server.port 8501 --server.enableCORS false

thread = threading.Thread(target=run_streamlit)
thread.start()

# Ouvrir un tunnel ngrok
public_url = ngrok.connect(8501)
print("🌐 Accédez au dashboard via :", public_url)

🌐 Accédez au dashboard via : NgrokTunnel: "https://haughty-elroy-comprehensibly.ngrok-free.dev" -> "http://localhost:8501"
